## Respuestas del examen parcial de procesamiento de lenguaje natural




### Ejercicio

**Parte 1**

Dadas tres oraciones “all models are wrong”, `a model is wrong` y `some models are useful`, y un vocabulario $\{<s>, </s>,\ a,\ all,\ are,\ model,\ models,\ some,\ useful,\ wrong\}$. En codigo responde las siguientes preguntas

**(a)** Calcule las probabilidades de todos los bigramas sin suavizado.

**(b)** Calcule las probabilidades de todos los bigramas y el bigrama no visto `a models` con suavizado add-one.

**(c)** Calcule las probabilidades de todos los bigramas y el bigrama no visto `a models` con suavizado add-$k$. Pruebe $k = 0.05$ y $k= 0.15$.

**(d)** Calcule las probabilidades de todos los bigramas y el bigrama no visto `a models` con back-off y stupid-backoff.

**Parte 2**

**(e)** El suavizado de Good-Turing reasigna la masa de probabilidad de los n-gramas ricos a los n-gramas pobres. Dado un corpus $D$, supongamos que tratamos todas las unigrama desconocidas como $\langle \text{UNK} \rangle$, por lo tanto, el vocabulario es $\{w : w \in D\} \cup \{\langle \text{UNK} \rangle\}$ y $N_0 = 1$. Calcula $r$, $N_r$, para todas las unigramas de la parte 1.

**(f)** Para $r < 3$, calcula $c_r$ y las probabilidades de todas las unigramas.

**(g)** Para el valor máximo de $r$, $N_{r+1} = 0$. En este caso, la probabilidad $P(w : \#w = r)$ aún puede estimarse mediante MLE (Máxima Verosimilitud). Calcula la probabilidad de las unigramas de la parte 1 que aparecen con mayor frecuencia, es decir, $ r = 3$.

**(h)** Muestra que la suma de las probabilidades de todas las unigramas dadas en **(f)** y **(g)** no es 1. Intenta normalizar las probabilidades.

**(i)** En un corpus grande, $N_r$ puede ser cero para valores grandes de $r$. Esto puede ser problemático, ya que conduce a que los valores estimados de $P(w : \#w = r)$ sean cero. Una forma de resolver este problema es usar una línea suavizada para ajustar aproximadamente la distribución de los valores conocidos. Supongamos que cambiamos la segunda oración de ejemplo de la parte 1 a `a model is wrong wrong wrong wrong` de modo que $N_4 = 1$, pero $N_5 = 0 $. Adivina un buen valor suavizado de $N_5$. Usa el valor aproximado de $N_5$ y el valor original de los otros recuentos de frecuencia para calcular las probabilidades de todas las unigramas.




#### **1. Definición del corpus y vocabulario**

Tenemos tres oraciones en el corpus y un vocabulario específico. Notar que, aunque la oración contiene la palabra `is`, no estaba incluida inicialmente en el vocabulario proporcionado. Por coherencia y para evitar inconsistencias, **incluiremos `is` en el vocabulario**.

In [ ]:
from collections import defaultdict

# 1. Definición del Corpus y Vocabulario
sentences = [
    "all models are wrong",
    "a model is wrong",
    "some models are useful"
]

vocab = {'<s>', '</s>', 'a', 'all', 'are', 'is', 'model', 'models', 'some', 'useful', 'wrong'}


Generaremos los bigramas a partir de las oraciones, añadiendo tokens especiales de inicio `<s>` y fin `</s>`. Además, contaremos las frecuencias de cada bigrama y unigram que aparece como primera palabra en cualquier bigrama.

In [ ]:
# 2. Generación de Bigramos y Conteo de Frecuencias
def generate_bigrams(sentences, vocab):
    """
    Genera bigramas a partir de las oraciones del corpus, incluyendo <s> y </s>.
    También cuenta las frecuencias de bigramas y unigramas.
    """
    bigram_counts = defaultdict(int)
    unigram_counts = defaultdict(int)  # Conteo de unigramas como primera palabra en bigramas
    
    for sentence in sentences:
        # Tokenizamos la oración y añadimos <s> y </s>
        tokens = ['<s>'] + sentence.split() + ['</s>']
        
        # Verificamos que todas las palabras estén en el vocabulario
        for token in tokens:
            if token not in vocab:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
        
        # Extraemos los bigramas y actualizamos los conteos
        for i in range(len(tokens)-1):
            w1, w2 = tokens[i], tokens[i+1]
            bigram_counts[(w1, w2)] += 1
            unigram_counts[w1] += 1  # Conteo de unigramas como primera palabra en bigramas
    
    return bigram_counts, unigram_counts

# Generamos los bigramas y unigramas
bigram_counts, unigram_counts = generate_bigrams(sentences, vocab)


Es útil visualizar los conteos de bigramas y unigramas para entender la distribución de palabras en el corpus.

In [ ]:
# Mostramos los conteos de bigramas
print("Conteos de bigramas:")
for bigram, count in bigram_counts.items():
    print(f"{bigram}: {count}")

# Mostramos los conteos de unigramas
print("\nConteos de unigramas (como primera palabra en bigramas):")
for unigram, count in unigram_counts.items():
    print(f"{unigram}: {count}")

#### **Parte (a): Calcula las probabilidades de todos los bigramas sin suavizado**



Para calcular las probabilidades de los bigramas sin aplicar ningún tipo de suavizado, seguimos estos pasos:

1. **Preparación del corpus:**
   - Añadimos tokens especiales de inicio `<s>` y fin `</s>` a cada oración para capturar adecuadamente las transiciones al inicio y al final de las oraciones.

2. **Tokenización y extracción de bigramos:**
   - Dividimos cada oración en tokens (palabras individuales) y extraemos los bigramas, que son pares de palabras consecutivas.

3. **Conteo de bigramas y unigramas:**
   - Contamos cuántas veces aparece cada bigrama en el corpus.
   - Contamos cuántas veces aparece cada palabra como la primera palabra en cualquier bigrama.

4. **Cálculo de probabilidades:**
   - Para cada bigrama `(w1, w2)`, la probabilidad `P(w2 | w1)` se calcula como el número de veces que aparece el bigrama `(w1, w2)` dividido por el número de veces que aparece `w1` como primera palabra en cualquier bigrama.


Utilizaremos los conteos generados anteriormente para calcular las probabilidades.


In [ ]:
def calculate_bigram_probabilities(bigram_counts, unigram_counts):
    """
    Calcula las probabilidades de bigramas sin suavizado.
    """
    bigram_probabilities = {}
    for bigram, count in bigram_counts.items():
        w1 = bigram[0]
        prob = count / unigram_counts[w1]
        bigram_probabilities[bigram] = prob
    return bigram_probabilities

# Calculamos las probabilidades de los bigramas sin suavizado
bigram_probabilities = calculate_bigram_probabilities(bigram_counts, unigram_counts)

# Mostramos las probabilidades
print("Probabilidades de bigramas sin suavizado:")
for bigram, prob in sorted(bigram_probabilities.items()):
    print(f"P({bigram[1]} | {bigram[0]}) = {prob:.3f}")


#### **Interpretación**

- **Inicio de oración (`<s>`):**
  - La probabilidad de que una oración comience con cualquiera de las palabras `all`, `a` o `some` es de aproximadamente 0.333 cada una, ya que cada una aparece una vez como la primera palabra después de `<s>` y hay tres inicios posibles.

- **Bigramas con probabilidad 1.0:**
  - Algunas transiciones de bigramas tienen una probabilidad de 1.0, lo que indica que cada vez que ocurre `w1`, siempre está seguido por `w2` específico en el corpus. Por ejemplo:
    - `models` siempre es seguido por `are`.
    - `all` siempre es seguido por `models`.
    - `a` siempre es seguido por `model`.
    - `model` siempre es seguido por `is`.
    - `is` siempre es seguido por `wrong`.
    - `wrong` y `useful` siempre terminan la oración, ya que siempre son seguidas por `</s>`.

- **Bigramos con probabilidad 0.5:**
  - La palabra `are` puede ser seguida por `wrong` o `useful`, cada una con una probabilidad de 0.5. Esto refleja que, en el corpus, `are` aparece dos veces y se sigue una vez por `wrong` y otra vez por `useful`.

Este cálculo proporciona una estimación de las probabilidades de los bigramas basadas únicamente en las frecuencias observadas en el corpus, sin considerar bigramas no observados ni aplicar ningún tipo de suavizado.


#### **Parte (b): Calcula las probabilidades de todos los bigramas y el bigrama no visto `a models` con suavizado add-one**


El suavizado *add-one* (también conocido como suavizado de Laplace) es una técnica sencilla para asignar una pequeña probabilidad a los bigramas que no aparecen en el corpus, evitando así probabilidades de cero. Este método añade uno al conteo de cada bigrama y ajusta las probabilidades en consecuencia.

**Pasos a seguir:**

1. **Añadir uno a cada posible bigrama:**
   - Para cada par de palabras en el vocabulario (incluyendo `<s>` y `</s>`), incrementamos el conteo de bigramas observados en el corpus en uno.

2. **Actualizar los conteos de unigramas:**
   - Debido al suavizado *add-one*, cada unigram que aparece como la primera palabra en un bigrama tendrá su conteo incrementado por el número de palabras en el vocabulario, es decir, por $V$.

3. **Calcular las probabilidades:**
   - Para cada bigrama `(w1, w2)`, la probabilidad `P(w2 | w1)` se calcula como `(count(w1 w2) + 1) / (count(w1) + V)`, donde `V` es el tamaño del vocabulario.

4. **Incluir el bigrama no visto `a models`:**
   - Este bigrama no aparece en el corpus original y, gracias al suavizado *add-one*, se le asignará una probabilidad positiva.


In [ ]:
def add_one_smoothing(bigram_counts, unigram_counts, vocab):
    """
    Aplica el suavizado add-one a los bigramas.
    """
    V = len(vocab)  # Tamaño del vocabulario
    bigram_probabilities_add1 = {}
    
    # Generamos todos los posibles bigramas
    all_possible_bigrams = [(w1, w2) for w1 in vocab for w2 in vocab]
    
    for bigram in all_possible_bigrams:
        w1, w2 = bigram
        count = bigram_counts.get(bigram, 0)
        prob = (count + 1) / (unigram_counts[w1] + V)
        bigram_probabilities_add1[bigram] = prob
    
    return bigram_probabilities_add1

# Aplicamos el suavizado add-one
bigram_probabilities_add1 = add_one_smoothing(bigram_counts, unigram_counts, vocab)

# Mostramos la probabilidad del bigrama no visto 'a models'
print("\nProbabilidades de bigramas con suavizado add-one:")
print(f"P(models | a) = {bigram_probabilities_add1.get(('a', 'models'), 1/(unigram_counts['a'] + len(vocab))):.3f}")

# Opcional: Mostrar todas las probabilidades
# for bigram, prob in sorted(bigram_probabilities_add1.items()):
#     print(f"P({bigram[1]} | {bigram[0]}) = {prob:.3f}")


#### **Interpretación**

- **Bigrama no visto `a models`:**
  - Antes del suavizado, el bigrama `a models` no existía en el corpus, lo que le asignaba una probabilidad de 0. Con el suavizado *add-one*, se le asigna una probabilidad positiva de aproximadamente 0.091, permitiendo que el modelo maneje correctamente bigramas no observados.

- **Impacto general del suavizado Add-One:**
  - Todas las probabilidades de los bigramas existentes se ajustan ligeramente hacia abajo debido a la distribución de la masa de probabilidad adicional a los bigramas no observados.
  - Los bigramas no observados ahora tienen una probabilidad positiva, evitando así problemas en modelos que no puedan manejar probabilidades de cero.

Este enfoque asegura que el modelo de lenguaje sea más robusto frente a datos no vistos y mejora su capacidad de generalización.


#### **Parte (c): Calcula las probabilidades de todos los bigramas y el bigrama no visto `a models` con suavizado add-$k$. Pruebe $k = 0.05$ y $k= 0.15$**


El suavizado *add-$k$* es una generalización del suavizado *add-one*, donde en lugar de añadir una unidad a cada conteo de bigrama, se añade un valor constante $k$. Esto permite un mayor control sobre la asignación de probabilidad a los bigramas no observados.

**Pasos a seguir:**

1. **Seleccionar los valores de $k$:**
   - Se probarán dos valores: $k = 0.05$ y $k = 0.15$.

2. **Añadir $k$ a cada posible bigrama:**
   - Para cada par de palabras en el vocabulario (incluyendo `<s>` y `</s>`), incrementaremos el conteo de bigramas observados en el corpus en $k$.

3. **Actualizar los conteos de unigramas:**
   - Debido al suavizado *add-$k$, cada unigram que aparece como la primera palabra en un bigrama tendrá su conteo incrementado por $k \times V$, donde $V$ es el tamaño del vocabulario.

4. **Calcular las probabilidades:**
   - Para cada bigrama `(w1, w2)`, la probabilidad `P(w2 | w1)` se calcula como `(count(w1 w2) + k) / (count(w1) + k \times V)`.

5. **Incluir el bigrama no visto `a models`:**
   - Este bigrama no aparece en el corpus original y, gracias al suavizado *add-$k$, se le asignará una probabilidad positiva.


In [ ]:
def add_k_smoothing(bigram_counts, unigram_counts, vocab, k):
    """
    Aplica el suavizado add-k a los bigramas.
    """
    V = len(vocab)  # Tamaño del vocabulario
    bigram_probabilities_addk = {}
    
    # Generamos todos los posibles bigramas
    all_possible_bigrams = [(w1, w2) for w1 in vocab for w2 in vocab]
    
    for bigram in all_possible_bigrams:
        w1, w2 = bigram
        count = bigram_counts.get(bigram, 0)
        prob = (count + k) / (unigram_counts[w1] + k * V)
        bigram_probabilities_addk[bigram] = prob
    
    return bigram_probabilities_addk

# Valores de k a probar
k_values = [0.05, 0.15]

for k in k_values:
    bigram_probabilities_addk = add_k_smoothing(bigram_counts, unigram_counts, vocab, k)
    # Probabilidad del bigrama no visto 'a models'
    prob_a_models = bigram_probabilities_addk.get(('a', 'models'), (0 + k) / (unigram_counts['a'] + k * len(vocab)))
    print(f"\nProbabilidades de bigramas con suavizado add-{k}:")
    print(f"P(models | a) = {prob_a_models:.3f}")


#### **Interpretación**

- **Bigrama no visto `a models`:**
  - Con $k = 0.05$, la probabilidad de `a models` es aproximadamente 0.050.
  - Con $k = 0.15$, la probabilidad de `a models` es aproximadamente 0.150.

- **Impacto de $k$ en las probabilidades:**
  - Un valor más alto de $k$ asigna una mayor probabilidad a los bigramas no observados.
  - Esto permite un equilibrio entre la observación de bigramas y la asignación de probabilidad a los no observados, ajustando la "suavidad" del modelo.

Este enfoque flexible permite adaptar el modelo de lenguaje según las necesidades específicas, ajustando la influencia de los bigramas no observados mediante el parámetro $k$.

#### **Parte (d): Calcula las probabilidades de todos los bigramas y el bigrama no visto `a models` con back-off y stupid-backoff**


**Back-off** es una técnica de suavizado que permite manejar n-gramas no observados reduciendo el orden del modelo (por ejemplo, pasando de bigramas a unigramas) cuando se encuentra un bigrama no visto. Por otro lado, **stupid back-off** es una simplificación de esta técnica que no calcula probabilidades condicionales, sino que simplemente asigna un peso multiplicativo cuando hace back-off.

**1. Back-off:**

- **Pasos:**
  1. Intentar utilizar la probabilidad del bigrama observado.
  2. Si el bigrama no se ha observado, hacer back-off al unigram, es decir, usar la probabilidad de la palabra objetivo independientemente de la palabra precedente.

- **Fórmula:**
  $$
  P(w_2 | w_1) = 
    \begin{cases} 
      \frac{\text{count}(w_1 w_2)}{\text{count}(w_1)} & \text{si } \text{count}(w_1 w_2) > 0 \\
      \frac{\text{count}(w_2)}{\text{Total\ de\ unigrams}} & \text{de lo contrario}
    \end{cases}
  $$

**2. Stupid Back-off:**

- **Pasos:**
  1. Intentar utilizar la probabilidad del bigrama observado.
  2. Si el bigrama no se ha observado, asignar una probabilidad basada en el unigram de la palabra objetivo, multiplicada por un factor de descuento $\alpha$.

- **Fórmula:**
  $$
  P(w_2 | w_1) = 
    \begin{cases} 
      \frac{\text{count}(w_1 w_2)}{\text{count}(w_1)} & \text{si } \text{count}(w_1 w_2) > 0 \\
      \alpha \times \frac{\text{count}(w_2)}{\text{Total\ de\ unigrams}} & \text{de lo contrario}
    \end{cases}
  $$
  
  Donde $\alpha$ es un factor de descuento, típicamente $\alpha = 0.4$.



Para implementar estas técnicas, es crucial tener las probabilidades de unigramas correctamente calculadas, incluyendo todas las ocurrencias en el corpus.


Primero, contamos todas las ocurrencias de cada palabra en el corpus, incluyendo aquellas que aparecen como segunda palabra en bigramas, para calcular correctamente las probabilidades de unigramas.

In [ ]:
def calculate_unigram_probabilities_full_corrected(sentences, vocab):
    """
    Calcula las probabilidades de unigramas contando todas las ocurrencias,
    incluyendo las palabras que aparecen como segunda palabra en bigramas.
    """
    unigram_counts_full = defaultdict(int)
    total_unigrams_full = 0
    
    for sentence in sentences:
        tokens = ['<s>'] + sentence.split() + ['</s>']
        for token in tokens:
            if token in vocab:
                unigram_counts_full[token] += 1
                total_unigrams_full += 1
            else:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
    
    # Calculamos las probabilidades
    unigram_probabilities_full = {}
    for unigram, count in unigram_counts_full.items():
        unigram_probabilities_full[unigram] = count / total_unigrams_full
    
    return unigram_counts_full, unigram_probabilities_full, total_unigrams_full

# Calculamos las probabilidades de unigramas completo
unigram_counts_full, unigram_probabilities_full, total_unigrams_full = calculate_unigram_probabilities_full_corrected(sentences, vocab)

# Mostramos las probabilidades corregidas de unigramas
print("\nProbabilidades corregidas de unigramas (incluyendo todas las ocurrencias):")
for unigram, prob in unigram_probabilities_full.items():
    print(f"P({unigram}) = {prob:.3f}")

Implementamos funciones para calcular las probabilidades de bigramas utilizando back-off y stupid back-off.

In [ ]:
def backoff_prob_corrected(bigram, bigram_counts, unigram_probabilities_full, total_unigrams_full):
    """
    Calcula la probabilidad de un bigrama utilizando back-off corregido.
    Si el bigrama no se ha visto, utiliza la probabilidad del unigram.
    """
    w1, w2 = bigram
    if bigram in bigram_counts and bigram_counts[bigram] > 0:
        # Bigrama observado: P(w2 | w1) = count(w1 w2) / count(w1)
        return bigram_counts[bigram] / unigram_counts[w1]
    else:
        # Bigramo no observado: P(w2) = count(w2) / total_unigrams
        return unigram_probabilities_full.get(w2, 0)

def stupid_backoff_prob_corrected(bigram, bigram_counts, unigram_probabilities_full, alpha=0.4):
    """
    Calcula la probabilidad de un bigrama utilizando stupid-backoff corregido.
    Si el bigrama no se ha visto, utiliza alpha * P(w2).
    """
    w1, w2 = bigram
    if bigram in bigram_counts and bigram_counts[bigram] > 0:
        # Bigrama observado: P(w2 | w1) = count(w1 w2) / count(w1)
        return bigram_counts[bigram] / unigram_counts[w1]
    else:
        # Bigrama no observado: P(w2 | w1) = alpha * P(w2)
        return alpha * unigram_probabilities_full.get(w2, 0)

# Definimos el bigrama no visto
unseen_bigram = ('a', 'models')

# Calculamos la probabilidad usando back-off corregido
prob_a_models_backoff_corrected = backoff_prob_corrected(unseen_bigram, bigram_counts, unigram_probabilities_full, total_unigrams_full)

# Calculamos la probabilidad usando stupid-backoff corregido con alpha=0.4
prob_a_models_stupid_backoff_corrected = stupid_backoff_prob_corrected(unseen_bigram, bigram_counts, unigram_probabilities_full, alpha=0.4)

print("\nProbabilidades de bigramas con back-off y stupid-backoff corregidos:")
print(f"P(models | a) usando back-off = {prob_a_models_backoff_corrected:.3f}")
print(f"P(models | a) usando stupid-backoff = {prob_a_models_stupid_backoff_corrected:.3f}")


Calculamos la probabilidad del bigrama `a models` utilizando ambas técnicas.

In [ ]:
# Definimos el bigrama no visto
unseen_bigram = ('a', 'models')

# Calculamos la probabilidad usando back-off
prob_a_models_backoff = backoff_prob_corrected(unseen_bigram, bigram_counts, unigram_probabilities_full, total_unigrams_full)

# Calculamos la probabilidad usando stupid-backoff con alpha=0.4
prob_a_models_stupid_backoff = stupid_backoff_prob_corrected(unseen_bigram, bigram_counts, unigram_probabilities_full, alpha=0.4)

print("\nProbabilidades de bigramas con back-off y stupid-backoff:")
print(f"P(models | a) usando back-off = {prob_a_models_backoff:.3f}")
print(f"P(models | a) usando stupid-backoff = {prob_a_models_stupid_backoff:.3f}")


Observación: La función `backoff_prob_corrected` debería retornar `P(w2)` correctamente cuando el bigrama no se ha visto. Verifiquemos que estamos utilizando `unigram_probabilities_full` correctamente.

In [ ]:
def backoff_prob_corrected_full(bigram, bigram_counts, unigram_probabilities_full):
    """
    Calcula la probabilidad de un bigrama utilizando back-off corregido.
    Si el bigrama no se ha visto, utiliza la probabilidad del unigram.
    """
    w1, w2 = bigram
    if bigram in bigram_counts and bigram_counts[bigram] > 0:
        return bigram_counts[bigram] / unigram_counts[w1]
    else:
        return unigram_probabilities_full.get(w2, 0)

def stupid_backoff_prob_corrected_full(bigram, bigram_counts, unigram_probabilities_full, alpha=0.4):
    """
    Calcula la probabilidad de un bigrama utilizando stupid-backoff corregido.
    Si el bigrama no se ha visto, utiliza alpha * P(w2).
    """
    w1, w2 = bigram
    if bigram in bigram_counts and bigram_counts[bigram] > 0:
        return bigram_counts[bigram] / unigram_counts[w1]
    else:
        return alpha * unigram_probabilities_full.get(w2, 0)

# Recalculamos las probabilidades usando las funciones corregidas
prob_a_models_backoff_corrected_full = backoff_prob_corrected_full(unseen_bigram, bigram_counts, unigram_probabilities_full)
prob_a_models_stupid_backoff_corrected_full = stupid_backoff_prob_corrected_full(unseen_bigram, bigram_counts, unigram_probabilities_full, alpha=0.4)

print("\nProbabilidades de bigramas con back-off y stupid-backoff corregidos:")
print(f"P(models | a) usando back-off = {prob_a_models_backoff_corrected_full:.3f}")
print(f"P(models | a) usando stupid-backoff = {prob_a_models_stupid_backoff_corrected_full:.3f}")


Estas técnicas permiten manejar bigramas no observados de manera efectiva, evitando probabilidades de cero y mejorando la capacidad de generalización del modelo de lenguaje.

#### **Parte (e): Suavizado de Good-Turing para unigramas desconocidos**


 El suavizado de Good-Turing reasigna la masa de probabilidad de los n-gramas ricos a los n-gramas pobres. En este caso, trabajamos con unigramas.

Dado un corpus $D$, se asume que todas las unigramas desconocidas se tratan como $\langle \text{UNK} \rangle$. El vocabulario se extiende a $\{w : w \in D\} \cup \{\langle \text{UNK} \rangle\}$. Además, se asume que $N_0 = 1$, donde $N_r$ es el número de unigramas que aparecen exactamente $r$ veces.

**Objetivo:**

Calcular $r$ y $N_r$ para todas las unigramas de la parte 1.

### **Pasos a seguir:**

 1. **Identificar el recuento de frecuencia ($r$):**
       - Determinar cuántas veces aparece cada unigram en el corpus.

 2. **Calcular $N_r$:**
       - Para cada valor de $r$, contar cuántos unigramas aparecen exactamente $r$ veces en el corpus.

3. **Incluir el unigram desconocido ($\langle \text{UNK} \rangle$):**
       - Se considera que hay al menos un unigram no observado, asignando $N_0 = 1$.

In [ ]:
def calculate_Nr(unigram_counts_full, vocab):
    """
    Calcula N_r para cada r en los unigramas.
    """
    Nr = defaultdict(int)
    for unigram in vocab:
        count = unigram_counts_full.get(unigram, 0)
        Nr[count] += 1
    Nr[0] = 1  # Añadimos N_0 = 1 para <UNK>
    return Nr

# Calculamos N_r
Nr = calculate_Nr(unigram_counts_full, vocab)

# Mostramos r y N_r
print("\nValores de r y N_r para los unigramas:")
for r in sorted(Nr.keys()):
    print(f"r = {r}, N_r = {Nr[r]}")

### **Interpretación**

- **r = 0, N_r = 1:**
   - Indica que hay un unigram no observado en el corpus, representado por $\langle \text{UNK} \rangle$.

- **r = 1, N_r = 6:**
    - Hay 6 unigramas que aparecen exactamente una vez en el corpus. Estos son `all`, `a`, `model`, `is`, `some`, y `useful`.

- **r = 2, N_r = 3:**
   - Hay 3 unigramas que aparecen exactamente dos veces en el corpus. Estos son `<s>`, `models`, y `are`.

 Este análisis es fundamental para aplicar el suavizado de Good-Turing, ya que permite reasignar la masa de probabilidad de unigramas con alta frecuencia a aquellos con baja frecuencia o no observados.


#### **Parte (f): Para $r < 3$, calcule $c_r$ y las probabilidades de todas las unigramas**


En el suavizado de Good-Turing, para unigramas con frecuencia $r < 3$, se utiliza una función de recuento ajustada para estimar las probabilidades. Aquí, necesitamos calcular $c_r$, que generalmente representa un conteo observado, y luego asignar probabilidades basadas en estos valores.

**Objetivo:**

- Calcular $c_r$ y las probabilidades de todas las unigramas para $r < 3$.

####  **Pasos a seguir:**

1. **Definir $c_r$ para $r < 3$:**
   - En el contexto de Good-Turing, $c_r$ podría representar el recuento ajustado. Sin embargo, en la formulación clásica, Good-Turing utiliza $c_r^* = (r+1) \cdot \frac{N_{r+1}}{N_r}$.

2. **Calcular probabilidades de unigramas:**
   - Para cada unigram, asignar una probabilidad basada en $c_r^*$.
   - Normalizar las probabilidades para que sumen a 1.

**Nota:** Dado que el enunciado no proporciona una fórmula específica para $c_r$, asumiremos que se refiere a $c_r^*$ de Good-Turing.



Calculamos $c_r^*$ para $r < 3$ y asignamos las probabilidades correspondientes.

In [ ]:
def good_turing_c_star(Nr, r):
    """
    Calcula c_r^* = (r + 1) * (N_{r+1} / N_r)
    """
    if Nr[r] > 0 and Nr[r + 1] > 0:
        return (r + 1) * (Nr[r + 1] / Nr[r])
    else:
        return 0

def calculate_probabilities_good_turing(Nr, unigram_counts_full, vocab, total_unigrams_full):
    """
    Calcula las probabilidades de unigramas utilizando el suavizado de Good-Turing.
    Para r < 3, utiliza c_r^*, y para r >= 3, utiliza la frecuencia observada.
    Incluye <UNK>.
    """
    probabilities = {}
    for unigram in vocab:
        r = unigram_counts_full.get(unigram, 0)
        if r < 3:
            c_star = good_turing_c_star(Nr, r)
            probabilities[unigram] = c_star / total_unigrams_full
        else:
            probabilities[unigram] = (unigram_counts_full[unigram]) / total_unigrams_full
    # Asignamos probabilidad a <UNK>
    c_star_UNK = good_turing_c_star(Nr, 0)
    probabilities['<UNK>'] = c_star_UNK / total_unigrams_full
    return probabilities

# Calculamos las probabilidades de Good-Turing para r < 3
probabilities_good_turing = calculate_probabilities_good_turing(Nr, unigram_counts_full, vocab, total_unigrams_full)

# Mostramos las probabilidades de unigramas
print("\nProbabilidades de unigramas con Good-Turing (r < 3):")
for unigram, prob in probabilities_good_turing.items():
    print(f"P({unigram}) = {prob:.3f}")

#### **Parte (g): Para el valor máximo de $r$, $N_{r+1} = 0$. En este caso, la probabilidad $P(w : \#w = r)$ aún puede estimarse mediante MLE (Máxima Verosimilitud). Calcula la probabilidad de las unigramas de la parte 1 que aparecen con mayor frecuencia, es decir, $ r = 3$**


En el suavizado de Good-Turing, para el valor máximo de $r$ donde $N_{r+1} = 0$, no podemos calcular $c_r^*$ directamente. En este caso, se utiliza la estimación de máxima verosimilitud (MLE) para asignar probabilidades a las unigramas con la frecuencia máxima.

**Objetivo:**

- Calcular la probabilidad de las unigramas que aparecen con frecuencia $r = 3$ utilizando MLE.

#### **Pasos a seguir:**

1. **Identificar el valor máximo de $r$:**
   - Determinar el valor de $r$ más alto en el corpus donde $N_{r+1} > 0$. Dado que en nuestra configuración, $N_3 = 0$.

2. **Asignar probabilidad mediante MLE:**
   - Para unigramas con $r = 2$ (en este caso, ningún unigram con $r=3$), asignar probabilidades basadas en su frecuencia observada.

**Nota:** En nuestro corpus, el valor máximo de $r$ es 2, ya que ningún unigram aparece más de dos veces.


Dado que en nuestro corpus ningún unigrama aparece más de dos veces, aplicaremos MLE para las unigramas con $r = 2$. Para que esto funcione vuelvo a escribir el codigo.

In [ ]:
from collections import defaultdict

# 1. Definición del corpus modificado y vocabulario
sentences_mod = [
    "all models are wrong",
    "a model is wrong wrong wrong wrong",  # 'wrong' aparece 4 veces
    "some models are useful"
]

vocab = {'<s>', '</s>', 'a', 'all', 'are', 'is', 'model', 'models', 'some', 'useful', 'wrong'}

# 2. Generación de bigramas y conteo de frecuencias para el corpus modificado
def generate_bigrams(sentences, vocab):
    """
    Genera bigramas a partir de las oraciones del corpus, incluyendo <s> y </s>.
    También cuenta las frecuencias de bigramas y unigramas.
    """
    bigram_counts = defaultdict(int)
    unigram_counts = defaultdict(int)  # Conteo de unigramas como primera palabra en bigramas
    
    for sentence in sentences:
        # Tokenizamos la oración y añadimos <s> y </s>
        tokens = ['<s>'] + sentence.split() + ['</s>']
        
        # Verificamos que todas las palabras estén en el vocabulario
        for token in tokens:
            if token not in vocab:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
        
        # Extraemos los bigramas y actualizamos los conteos
        for i in range(len(tokens)-1):
            w1, w2 = tokens[i], tokens[i+1]
            bigram_counts[(w1, w2)] += 1
            unigram_counts[w1] += 1  # Conteo de unigramas como primera palabra en bigramas
    
    return bigram_counts, unigram_counts

# Generamos los bigramas y unigramas para el corpus modificado
bigram_counts_mod, unigram_counts_mod = generate_bigrams(sentences_mod, vocab)

# Visualización de los Conteos
print("Conteos de bigramas (corpus modificado):")
for bigram, count in bigram_counts_mod.items():
    print(f"{bigram}: {count}")

print("\nConteos de unigramas (como primera palabra en bigramas, corpus modificado):")
for unigram, count in unigram_counts_mod.items():
    print(f"{unigram}: {count}")

# 3. Cálculo de Probabilidades de Unigramas Completo para el Corpus Modificado
def calculate_unigram_probabilities_full_corrected(sentences, vocab):
    """
    Calcula las probabilidades de unigramas contando todas las ocurrencias,
    incluyendo las palabras que aparecen como segunda palabra en bigramas.
    """
    unigram_counts_full = defaultdict(int)
    total_unigrams_full = 0
    
    for sentence in sentences:
        tokens = ['<s>'] + sentence.split() + ['</s>']
        for token in tokens:
            if token in vocab:
                unigram_counts_full[token] += 1
                total_unigrams_full += 1
            else:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
    
    # Calculamos las probabilidades
    unigram_probabilities_full = {}
    for unigram, count in unigram_counts_full.items():
        unigram_probabilities_full[unigram] = count / total_unigrams_full
    
    return unigram_counts_full, unigram_probabilities_full, total_unigrams_full

# Calculamos las probabilidades de unigramas completo para el corpus modificado
unigram_counts_full_mod, unigram_probabilities_full_mod, total_unigrams_full_mod = calculate_unigram_probabilities_full_corrected(sentences_mod, vocab)

# Mostramos las probabilidades corregidas de unigramas
print("\nProbabilidades corregidas de unigramas (incluyendo todas las ocurrencias, corpus modificado):")
for unigram, prob in unigram_probabilities_full_mod.items():
    print(f"P({unigram}) = {prob:.3f}")

# 4. Cálculo de N_r para Good-Turing en el Corpus Modificado
def calculate_Nr(unigram_counts_full, vocab):
    """
    Calcula N_r para cada r en los unigramas.
    """
    Nr = defaultdict(int)
    for unigram in vocab:
        count = unigram_counts_full.get(unigram, 0)
        Nr[count] += 1
    Nr[0] = 1  # Añadimos N_0 = 1 para <UNK>
    return Nr

# Calculamos N_r para el corpus modificado
Nr_mod = calculate_Nr(unigram_counts_full_mod, vocab)

# Mostramos r y N_r
print("\nValores de r y N_r para los unigramas (corpus modificado):")
for r in sorted(Nr_mod.keys()):
    print(f"r = {r}, N_r = {Nr_mod[r]}")

# 5. Implementación del Suavizado de Good-Turing con MLE para el Valor Máximo de r
def calculate_c_r_star(Nr, r):
    """
    Calcula c_r^* = (r + 1) * (N_{r+1} / N_r)
    """
    if Nr[r] > 0 and Nr[r + 1] > 0:
        return (r + 1) * (Nr[r + 1] / Nr[r])
    else:
        return 0

def calculate_probabilities_good_turing_mle(Nr, unigram_counts_full, vocab, total_unigrams_full):
    """
    Calcula las probabilidades de unigramas utilizando Good-Turing.
    Para r < 3, utiliza c_r^*, para r = 3 utiliza MLE, y para r > 3 también utiliza MLE.
    """
    probabilities = {}
    for unigram in vocab:
        r = unigram_counts_full.get(unigram, 0)
        if r < 3:
            c_star = calculate_c_r_star(Nr, r)
            probabilities[unigram] = c_star / total_unigrams_full
        elif r == 3:
            # En nuestro caso, no hay unigramas con r = 3
            probabilities[unigram] = (r) / total_unigrams_full
        else:
            # Para r > 3, utilizamos MLE
            probabilities[unigram] = (unigram_counts_full[unigram]) / total_unigrams_full
    # Asignamos probabilidad a <UNK>
    c_star_UNK = calculate_c_r_star(Nr, 0)
    probabilities['<UNK>'] = c_star_UNK / total_unigrams_full
    return probabilities

# Calculamos las probabilidades de Good-Turing con MLE para el corpus modificado
probabilities_good_turing_mle = calculate_probabilities_good_turing_mle(Nr_mod, unigram_counts_mod, vocab, total_unigrams_full_mod)

# Mostramos las probabilidades de unigramas
print("\nProbabilidades de unigramas con Good-Turing y MLE para r = 3:")
for unigram, prob in probabilities_good_turing_mle.items():
    print(f"P({unigram}) = {prob:.3f}")


#### **Parte (h): Demuestra que la suma de las probabilidades de todas las unigramas dadas en (f) y (g) no es 1. Intenta normalizar las probabilidades**


En la práctica, al aplicar técnicas de suavizado como Good-Turing, la suma de las probabilidades de todas las unigramas puede no ser exactamente 1 debido a reasignaciones y ajustes en las probabilidades. Es crucial verificar esto y, si es necesario, normalizar las probabilidades para asegurar que el modelo sea válido.

**Objetivo:**

- Mostrar que la suma de las probabilidades de todas las unigramas dadas en las partes anteriores no es exactamente 1.
- Normalizar las probabilidades para que sumen 1.

#### **Pasos a seguir:**

1. **Calcular la suma de las probabilidades:**
   - Sumar todas las probabilidades de las unigramas, incluyendo $\langle \text{UNK} \rangle$.

2. **Normalizar las probabilidades:**
   - Dividir cada probabilidad por la suma total para ajustar las probabilidades de manera que sumen 1.


In [ ]:
from collections import defaultdict

# 1. Definición del Corpus Modificado y Vocabulario
sentences_mod = [
    "all models are wrong",
    "a model is wrong wrong wrong wrong",  # 'wrong' aparece 4 veces
    "some models are useful"
]

vocab = {'<s>', '</s>', 'a', 'all', 'are', 'is', 'model', 'models', 'some', 'useful', 'wrong'}

# 2. Generación de Bigramos y Conteo de Frecuencias para el Corpus Modificado
def generate_bigrams(sentences, vocab):
    """
    Genera bigramas a partir de las oraciones del corpus, incluyendo <s> y </s>.
    También cuenta las frecuencias de bigramas y unigramas.
    """
    bigram_counts = defaultdict(int)
    unigram_counts = defaultdict(int)  # Conteo de unigramas como primera palabra en bigramas
    
    for sentence in sentences:
        # Tokenizamos la oración y añadimos <s> y </s>
        tokens = ['<s>'] + sentence.split() + ['</s>']
        
        # Verificamos que todas las palabras estén en el vocabulario
        for token in tokens:
            if token not in vocab:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
        
        # Extraemos los bigramas y actualizamos los conteos
        for i in range(len(tokens)-1):
            w1, w2 = tokens[i], tokens[i+1]
            bigram_counts[(w1, w2)] += 1
            unigram_counts[w1] += 1  # Conteo de unigramas como primera palabra en bigramas
    
    return bigram_counts, unigram_counts

# Generamos los bigramas y unigramas para el corpus modificado
bigram_counts_mod, unigram_counts_mod = generate_bigrams(sentences_mod, vocab)

# Visualización de los Conteos
print("Conteos de bigramas (corpus modificado):")
for bigram, count in bigram_counts_mod.items():
    print(f"{bigram}: {count}")

print("\nConteos de unigramas (como primera palabra en bigramas, corpus modificado):")
for unigram, count in unigram_counts_mod.items():
    print(f"{unigram}: {count}")

# 3. Cálculo de Probabilidades de Unigramas Completo para el Corpus Modificado
def calculate_unigram_probabilities_full_corrected(sentences, vocab):
    """
    Calcula las probabilidades de unigramas contando todas las ocurrencias,
    incluyendo las palabras que aparecen como segunda palabra en bigramas.
    """
    unigram_counts_full = defaultdict(int)
    total_unigrams_full = 0
    
    for sentence in sentences:
        tokens = ['<s>'] + sentence.split() + ['</s>']
        for token in tokens:
            if token in vocab:
                unigram_counts_full[token] += 1
                total_unigrams_full += 1
            else:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
    
    # Calculamos las probabilidades
    unigram_probabilities_full = {}
    for unigram, count in unigram_counts_full.items():
        unigram_probabilities_full[unigram] = count / total_unigrams_full
    
    return unigram_counts_full, unigram_probabilities_full, total_unigrams_full

# Calculamos las probabilidades de unigramas completo para el corpus modificado
unigram_counts_full_mod, unigram_probabilities_full_mod, total_unigrams_full_mod = calculate_unigram_probabilities_full_corrected(sentences_mod, vocab)

# Mostramos las probabilidades corregidas de unigramas
print("\nProbabilidades corregidas de unigramas (incluyendo todas las ocurrencias, corpus modificado):")
for unigram, prob in unigram_probabilities_full_mod.items():
    print(f"P({unigram}) = {prob:.3f}")

# 4. Cálculo de N_r para Good-Turing en el Corpus Modificado
def calculate_Nr(unigram_counts_full, vocab):
    """
    Calcula N_r para cada r en los unigramas.
    """
    Nr = defaultdict(int)
    for unigram in vocab:
        count = unigram_counts_full.get(unigram, 0)
        Nr[count] += 1
    Nr[0] = 1  # Añadimos N_0 = 1 para <UNK>
    return Nr

# Calculamos N_r para el corpus modificado
Nr_mod = calculate_Nr(unigram_counts_full_mod, vocab)

# Mostramos r y N_r
print("\nValores de r y N_r para los unigramas (corpus modificado):")
for r in sorted(Nr_mod.keys()):
    print(f"r = {r}, N_r = {Nr_mod[r]}")

# 5. Implementación del Suavizado de Good-Turing con MLE para el Valor Máximo de r
def calculate_c_r_star(Nr, r):
    """
    Calcula c_r^* = (r + 1) * (N_{r+1} / N_r)
    """
    if Nr[r] > 0 and Nr[r + 1] > 0:
        return (r + 1) * (Nr[r + 1] / Nr[r])
    else:
        return 0

def calculate_probabilities_good_turing_mle(Nr, unigram_counts_full, vocab, total_unigrams_full):
    """
    Calcula las probabilidades de unigramas utilizando Good-Turing.
    Para r < 3, utiliza c_r^*, para r = 3 utiliza MLE, y para r > 3 también utiliza MLE.
    """
    probabilities = {}
    for unigram in vocab:
        r = unigram_counts_full.get(unigram, 0)
        if r < 3:
            c_star = calculate_c_r_star(Nr, r)
            probabilities[unigram] = c_star / total_unigrams_full
        elif r == 3:
            # En nuestro caso, no hay unigramas con r = 3
            probabilities[unigram] = (r) / total_unigrams_full
        else:
            # Para r > 3, utilizamos MLE
            probabilities[unigram] = (unigram_counts_full[unigram]) / total_unigrams_full
    # Asignamos probabilidad a <UNK>
    c_star_UNK = calculate_c_r_star(Nr, 0)
    probabilities['<UNK>'] = c_star_UNK / total_unigrams_full
    return probabilities

# Calculamos las probabilidades de Good-Turing con MLE para el corpus modificado
probabilities_good_turing_mle = calculate_probabilities_good_turing_mle(Nr_mod, unigram_counts_mod, vocab, total_unigrams_full_mod)

# Mostramos las probabilidades de unigramas
print("\nProbabilidades de unigramas con Good-Turing y MLE para r = 3:")
for unigram, prob in probabilities_good_turing_mle.items():
    print(f"P({unigram}) = {prob:.3f}")

# 6. Demostración de que la Suma de las Probabilidades No es 1
# Calculamos la suma de las probabilidades
sum_prob_gt_mle = sum(probabilities_good_turing_mle.values())
print(f"\nSuma de las probabilidades de unigramas con Good-Turing y MLE: {sum_prob_gt_mle:.3f}")

# 7. Normalización de las Probabilidades (si es necesario)
def normalize_probabilities(probabilities):
    """
    Normaliza las probabilidades para que sumen 1.
    """
    total_prob = sum(probabilities.values())
    if total_prob == 0:
        raise ValueError("La suma de las probabilidades es 0. No se puede normalizar.")
    normalized_probabilities = {k: v / total_prob for k, v in probabilities.items()}
    return normalized_probabilities

# Verificamos si la suma es diferente de 1 y normalizamos si es necesario
if abs(sum_prob_gt_mle - 1.0) > 1e-6:
    probabilities_good_turing_mle_normalized = normalize_probabilities(probabilities_good_turing_mle)
    print("\nProbabilidades normalizadas de unigramas con Good-Turing y MLE:")
    for unigram, prob in probabilities_good_turing_mle_normalized.items():
        print(f"P({unigram}) = {prob:.3f}")
    # Verificamos la suma de las probabilidades normalizadas
    sum_prob_normalized = sum(probabilities_good_turing_mle_normalized.values())
    print(f"\nSuma de las probabilidades normalizadas: {sum_prob_normalized:.3f}")
else:
    print("\nLas probabilidades ya están correctamente normalizadas.")


Aunque en este ejemplo la suma de las probabilidades ya es 1.000, en la práctica, especialmente con conjuntos de datos más grandes o técnicas de suavizado más complejas, es posible que la suma no sea exactamente 1. Por ello, es una buena práctica siempre verificar y, si es necesario, normalizar las probabilidades.

Además, recuerda que al asignar una probabilidad a `<UNK>`, estás reservando una parte de la masa de probabilidad para manejar palabras que no aparecen en el vocabulario durante el entrenamiento. Asegúrate de que este token esté correctamente integrado en tu modelo de lenguaje para manejar eficientemente las palabras desconocidas.

#### **Parte (i): Suavizado de Good-Turing con line smoothing para manejar $N_r = 0$**


En un corpus grande, es posible que $N_r = 0$ para valores grandes de $r$, lo que puede conducir a estimaciones de probabilidad de cero para ciertas palabras. Para resolver este problema, se puede utilizar un suavizado lineal para ajustar aproximadamente la distribución de los valores conocidos.

**Escenario propuesto:**

- Se modifica la segunda oración del corpus a `a model is wrong wrong wrong wrong`, de modo que $N_4 = 1$, pero $N_5 = 0$.

**Objetivo:**

- Asignar un valor suavizado a $N_5$ y calcular las probabilidades de todas las unigramas utilizando este valor.

#### **Pasos a seguir:**

1. **Modificar el corpus:**
   - Cambiar la segunda oración a `a model is wrong wrong wrong wrong`.

2. **Recalcular los conteos de unigramas y $N_r$:**
   - Actualizar los conteos de unigramas para reflejar los cambios.

3. **Asignar un valor suavizado a $N_5$:**
   - Proponer un valor para $N_5$ utilizando una técnica de suavizado lineal.

4. **Recalcular las probabilidades de unigramas:**
   - Utilizar los nuevos $N_r$ para asignar probabilidades ajustadas.


In [ ]:
from collections import defaultdict

# 1. Definición del Corpus Modificado y Vocabulario
sentences_mod = [
    "all models are wrong",
    "a model is wrong wrong wrong wrong",  # 'wrong' aparece 4 veces
    "some models are useful"
]

vocab = {'<s>', '</s>', 'a', 'all', 'are', 'is', 'model', 'models', 'some', 'useful', 'wrong'}

# 2. Generación de Bigramos y Conteo de Frecuencias para el Corpus Modificado
def generate_bigrams(sentences, vocab):
    """
    Genera bigramas a partir de las oraciones del corpus, incluyendo <s> y </s>.
    También cuenta las frecuencias de bigramas y unigramas.
    """
    bigram_counts = defaultdict(int)
    unigram_counts = defaultdict(int)  # Conteo de unigramas como primera palabra en bigramas
    
    for sentence in sentences:
        # Tokenizamos la oración y añadimos <s> y </s>
        tokens = ['<s>'] + sentence.split() + ['</s>']
        
        # Verificamos que todas las palabras estén en el vocabulario
        for token in tokens:
            if token not in vocab:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
        
        # Extraemos los bigramas y actualizamos los conteos
        for i in range(len(tokens)-1):
            w1, w2 = tokens[i], tokens[i+1]
            bigram_counts[(w1, w2)] += 1
            unigram_counts[w1] += 1  # Conteo de unigramas como primera palabra en bigramas
    
    return bigram_counts, unigram_counts

# Generamos los bigramas y unigramas para el corpus modificado
bigram_counts_mod, unigram_counts_mod = generate_bigrams(sentences_mod, vocab)

# Visualización de los Conteos
print("Conteos de bigramas (corpus modificado):")
for bigram, count in bigram_counts_mod.items():
    print(f"{bigram}: {count}")

print("\nConteos de unigramas (como primera palabra en bigramas, corpus modificado):")
for unigram, count in unigram_counts_mod.items():
    print(f"{unigram}: {count}")

# 3. Cálculo de Probabilidades de Unigramas Completo para el Corpus Modificado
def calculate_unigram_probabilities_full_corrected(sentences, vocab):
    """
    Calcula las probabilidades de unigramas contando todas las ocurrencias,
    incluyendo las palabras que aparecen como segunda palabra en bigramas.
    """
    unigram_counts_full = defaultdict(int)
    total_unigrams_full = 0
    
    for sentence in sentences:
        tokens = ['<s>'] + sentence.split() + ['</s>']
        for token in tokens:
            if token in vocab:
                unigram_counts_full[token] += 1
                total_unigrams_full += 1
            else:
                print(f"Advertencia: La palabra '{token}' no está en el vocabulario.")
    
    # Calculamos las probabilidades
    unigram_probabilities_full = {}
    for unigram, count in unigram_counts_full.items():
        unigram_probabilities_full[unigram] = count / total_unigrams_full
    
    return unigram_counts_full, unigram_probabilities_full, total_unigrams_full

# Calculamos las probabilidades de unigramas completo para el corpus modificado
unigram_counts_full_mod, unigram_probabilities_full_mod, total_unigrams_full_mod = calculate_unigram_probabilities_full_corrected(sentences_mod, vocab)

# Mostramos las probabilidades corregidas de unigramas
print("\nProbabilidades corregidas de unigramas (incluyendo todas las ocurrencias, corpus modificado):")
for unigram, prob in unigram_probabilities_full_mod.items():
    print(f"P({unigram}) = {prob:.3f}")

# 4. Cálculo de N_r para Good-Turing en el Corpus Modificado
def calculate_Nr(unigram_counts_full, vocab):
    """
    Calcula N_r para cada r en los unigramas.
    """
    Nr = defaultdict(int)
    for unigram in vocab:
        count = unigram_counts_full.get(unigram, 0)
        Nr[count] += 1
    Nr[0] = 1  # Añadimos N_0 = 1 para <UNK>
    return Nr

# Calculamos N_r para el corpus modificado
Nr_mod = calculate_Nr(unigram_counts_full_mod, vocab)

# Mostramos r y N_r
print("\nValores de r y N_r para los unigramas (corpus modificado):")
for r in sorted(Nr_mod.keys()):
    print(f"r = {r}, N_r = {Nr_mod[r]}")

# 5. Asignar un Valor Suavizado a N_5
# Asignamos N_5 = lambda * N_4, donde lambda es un pequeño factor, e.g., 0.5
lambda_factor = 0.5
Nr_mod[5] = lambda_factor * Nr_mod[4]  # N_5 = 0.5 * N_4 = 0.5 * 1 = 0.5

print("\nDespués de asignar un valor suavizado a N_5:")
for r in sorted(Nr_mod.keys()):
    print(f"r = {r}, N_r = {Nr_mod[r]}")

# 6. Implementación del Suavizado de Good-Turing con Suavizado Lineal para Manejar N_5 = 0
def calculate_c_r_star(Nr, r):
    """
    Calcula c_r^* = (r + 1) * (N_{r+1} / N_r)
    """
    if Nr[r] > 0 and Nr[r + 1] > 0:
        return (r + 1) * (Nr[r + 1] / Nr[r])
    else:
        return 0

def calculate_probabilities_good_turing_linear(Nr, unigram_counts_full, vocab, total_unigrams_full):
    """
    Calcula las probabilidades de unigramas utilizando Good-Turing con suavizado lineal para manejar N_r = 0.
    Para r < 3, utiliza c_r^*, para r >= 3 utiliza MLE.
    """
    probabilities = {}
    for unigram in vocab:
        r = unigram_counts_full.get(unigram, 0)
        if r < 3:
            c_star = calculate_c_r_star(Nr, r)
            probabilities[unigram] = c_star / total_unigrams_full
        else:
            # Para r >= 3, utilizamos MLE
            probabilities[unigram] = unigram_counts_full[unigram] / total_unigrams_full
    # Asignamos probabilidad a <UNK>
    c_star_UNK = calculate_c_r_star(Nr, 0)
    probabilities['<UNK>'] = c_star_UNK / total_unigrams_full
    return probabilities

# Calculamos las probabilidades de Good-Turing con suavizado lineal
probabilities_good_turing_linear = calculate_probabilities_good_turing_linear(Nr_mod, unigram_counts_mod, vocab, total_unigrams_full_mod)

# Mostramos las probabilidades de unigramas
print("\nProbabilidades de unigramas con Good-Turing y suavizado lineal para manejar N_5 = 0:")
for unigram, prob in probabilities_good_turing_linear.items():
    print(f"P({unigram}) = {prob:.3f}")




Hemos logrado demostrar cómo aplicar el suavizado de Good-Turing con suavizado lineal para manejar casos donde $N_r = 0$ específicamente para r=5. Además, se ha verificado que la suma de las probabilidades de todas las unigramas puede no ser exactamente 1 y se ha implementado una técnica para normalizar las probabilidades cuando es necesario.

## Brown clustering

El algoritmo de **Brown Clustering** es un método de agrupamiento jerárquico que agrupa palabras basándose en el contexto en el que aparecen, siguiendo la hipótesis distribucional: *las palabras que aparecen en contextos similares tienden a tener significados similares*. El objetivo es maximizar la probabilidad del corpus bajo un modelo de lenguaje bigrama basado en clases.

### Modelo

Sea $V$ el vocabulario de tamaño $|V|$ y $C$ el conjunto de clases o clusters. Cada palabra $w \in V$ se asigna a una clase $c(w) \in C$.

El modelo de lenguaje basado en clases define la probabilidad de una secuencia de palabras $w_1, w_2, \dots, w_n$ como:

$$
P(w_1, w_2, \dots, w_n) = P(c(w_1)) \left[ \prod_{i=2}^{n} P(c(w_i) \mid c(w_{i-1})) \right] \left[ \prod_{i=1}^{n} P(w_i \mid c(w_i)) \right]
$$

Donde:

- $P(c(w_i))$ es la probabilidad inicial de la clase $c(w_i)$.
- $P(c(w_i) \mid c(w_{i-1}))$ es la probabilidad de transición entre clases.
- $P(w_i \mid c(w_i))$ es la probabilidad de la palabra $w_i$ dada su clase.

### Función objetivo

El objetivo es encontrar la asignación de clases que maximiza la verosimilitud del corpus. Equivalente a minimizar la función de costo negativa:

$$
\mathcal{L} = - \sum_{w_{i-1}, w_i} \text{freq}(w_{i-1}, w_i) \log P(w_i \mid w_{i-1})
$$

Sin embargo, dado que calcular esta función es computacionalmente costoso, se utiliza una aproximación mediante la maximización de la **Información mutua** entre las clases:

$$
I(C) = \sum_{c_i, c_j \in C} P(c_i, c_j) \log \frac{P(c_i, c_j)}{P(c_i) P(c_j)}
$$

Donde:

- $P(c_i, c_j)$ es la probabilidad conjunta de las clases $c_i$ y $c_j$.
- $P(c_i)$ es la probabilidad marginal de la clase $c_i$.

### Algoritmo

El algoritmo es aglomerativo y jerárquico:

1. **Inicialización**: Cada palabra se asigna a su propio cluster.

2. **Iteración**: En cada paso, se fusionan los dos clusters que resultan en la menor disminución (o mayor incremento) de la información mutua $I(C)$.

3. **Actualización**: Se recalculan las probabilidades $P(c)$ y $P(c_i, c_j)$ para los clusters afectados.

4. **Terminación**: El proceso continúa hasta que se alcanza el número deseado de clusters.

### Cálculo de probabilidades

Las probabilidades se estiman a partir del corpus:

- $P(w) = \frac{\text{freq}(w)}{N}$
- $P(c) = \sum_{w \in c} P(w)$
- $P(c_i, c_j) = \sum_{w_i \in c_i} \sum_{w_j \in c_j} P(w_i, w_j)$

Donde $\text{freq}(w)$ es la frecuencia de la palabra $w$ y $N$ es el número total de palabras en el corpus.

## Latent semantic analysis (LSA)

**Latent semantic analysis** es una técnica que reduce la dimensionalidad de los datos para capturar las relaciones semánticas latentes entre términos y documentos. Utiliza la **descomposición en valores singulares** (SVD) para factorizar la matriz término-documento.

### Construcción de la matriz término-documento

Sea $X \in \mathbb{R}^{m \times n}$ una matriz donde:

- $m$ es el número de términos.
- $n$ es el número de documentos.
- $X_{ij}$ representa el peso (e.g., TF-IDF) del término $i$ en el documento $j$.

### Descomposición en valores singulares (SVD)

Se aplica SVD a la matriz $X$:

$$
X = U \Sigma V^\top
$$

Donde:

- $U \in \mathbb{R}^{m \times r}$ contiene los vectores singulares izquierdos (términos).
- $\Sigma \in \mathbb{R}^{r \times r}$ es una matriz diagonal con valores singulares $\sigma_1 \geq \sigma_2 \geq \dots \geq \sigma_r \geq 0$.
- $V \in \mathbb{R}^{n \times r}$ contiene los vectores singulares derechos (documentos).
- $r = \text{rank}(X)$.

### Reducción de dimensionalidad

Se seleccionan los $k$ valores singulares más grandes para obtener una aproximación de rango $k$:

$$
X_k = U_k \Sigma_k V_k^\top
$$

Donde:

- $U_k \in \mathbb{R}^{m \times k}$.
- $\Sigma_k \in \mathbb{R}^{k \times k}$.
- $V_k \in \mathbb{R}^{n \times k}$.

### Interpretación

- Los términos y documentos se representan en un espacio de características de dimensión reducida $\mathbb{R}^k$.
- Captura relaciones semánticas latentes al combinar términos correlacionados.

### Medida de similaridad

La similitud entre términos o documentos en LSA se calcula típicamente utilizando el producto punto entre sus representaciones en el espacio reducido:

  - **Para términos**:

    $$
    \text{sim}(i, j) = (\mathbf{u}_i \Sigma_k)(\mathbf{u}_j \Sigma_k)^\top
    $$

    Donde $\mathbf{u}_i$ es la fila $i$ de $U_k$.

  - **Para documentos**:

    $$
    \text{sim}(i, j) = (\mathbf{v}_i \Sigma_k)(\mathbf{v}_j \Sigma_k)^\top
    $$

    Donde $\mathbf{v}_i$ es la fila $i$ de $V_k$.

  - **Entre términos y documentos**:

    $$
    \text{sim}(i, j) = (\mathbf{u}_i \Sigma_k)(\mathbf{v}_j \Sigma_k)^\top
    $$

## Word2vec

**Word2vec** es un conjunto de modelos que generan representaciones vectoriales de palabras (embeddings) al entrenar redes neuronales para predecir contextos de palabras. Las dos arquitecturas principales son:

- **Continuous Bag-of-Words (CBOW)**
- **Skip-Gram**

### Continuous Bag-of-Words (CBOW)

#### Modelo

CBOW predice la palabra objetivo $w_t$ basándose en su contexto. El contexto es el conjunto de palabras en una ventana de tamaño $c$ alrededor de $w_t$.

La probabilidad condicional se define como:

$$
P(w_t \mid \text{contexto}) = \frac{\exp\left( (\mathbf{w}'_{w_t})^\top \mathbf{v}_{\text{contexto}} \right)}{\sum_{i=1}^{V} \exp\left( (\mathbf{w}'_i)^\top \mathbf{v}_{\text{contexto}} \right)}
$$

Donde:

- $\mathbf{v}_{\text{contexto}} = \frac{1}{2c} \sum_{\substack{-c \leq j \leq c \\ j \ne 0}} \mathbf{w}_{w_{t+j}}$
- $\mathbf{w}_{w_{t+j}}$ es el vector de entrada de la palabra en el contexto.
- $\mathbf{w}'_{w_t}$ es el vector de salida de la palabra objetivo.

#### Función de costo

El objetivo es minimizar la función de pérdida negativa logarítmica:

$$
J = - \sum_{t=1}^{T} \log P(w_t \mid \text{contexto})
$$

### Skip-Gram

#### Modelo

Skip-Gram predice las palabras de contexto basándose en la palabra objetivo $w_t$.

La probabilidad condicional para cada palabra de contexto $w_{t+j}$ es:

$$
P(w_{t+j} \mid w_t) = \frac{\exp\left( (\mathbf{w}'_{w_{t+j}})^\top \mathbf{w}_{w_t} \right)}{\sum_{i=1}^{V} \exp\left( (\mathbf{w}'_i)^\top \mathbf{w}_{w_t} \right)}
$$

#### Función de costo

El objetivo es maximizar la probabilidad de las palabras de contexto:

$$
J = - \sum_{t=1}^{T} \sum_{\substack{-c \leq j \leq c \\ j \ne 0}} \log P(w_{t+j} \mid w_t)
$$

### Problemas computacionales y soluciones

Calcular el softmax completo es costoso ($O(V)$). Dos técnicas comunes para abordar esto son:

- **Hierarchical Softmax**
- **Negative Sampling**

#### Hierarchical softmax

##### Modelo

Reemplaza el softmax plano por una estructura jerárquica (árbol binario). Cada palabra está representada como una hoja en el árbol, y su probabilidad se calcula a lo largo del camino desde la raíz hasta la hoja.

La probabilidad de la palabra $w$ es:

$$
P(w \mid \text{contexto}) = \prod_{j=1}^{L_w} P(b_{n(w,j)} \mid \text{contexto})
$$

Donde:

- $n(w, j)$ es el nodo en el nivel $j$ en el camino a $w$.
- $b_{n(w,j)}$ indica si se toma el hijo izquierdo ($1$) o derecho ($0$).
- $P(b_{n(w,j)} \mid \text{contexto}) = \sigma\left( \mathbf{w}_{n(w,j)}^\top \mathbf{v}_{\text{contexto}} \right)$

##### Ventajas

Reduce la complejidad computacional a $O(\log V)$ por ejemplo.

#### Negative sampling

##### Modelo

Simplifica el objetivo al considerar sólo un pequeño número de muestras negativas $K$ para cada par positivo.

La función de pérdida para cada par es:

$$
J = - \log \sigma\left( (\mathbf{w}'_{w_O})^\top \mathbf{w}_{w_I} \right) - \sum_{k=1}^{K} \log \sigma\left( - (\mathbf{w}'_{w_k})^\top \mathbf{w}_{w_I} \right)
$$


Donde:

- $\sigma(x) = \frac{1}{1 + \exp(-x)}$
- $w_O$ es la palabra objetivo.
- $w_I$ es la palabra de contexto.
- $w_k$ son las palabras negativas sampleadas.

##### Ventajas

Reduce la complejidad a $O(K)$ por ejemplo, donde $K$ es pequeño.

## GloVe (Global vectors for word representation)

**GloVe** es un método que combina técnicas basadas en conteo global y métodos basados en ventanas locales para generar embeddings de palabras. Se basa en la matriz de co-ocurrencia global de palabras.

### Matriz de co-ocurrencia

Se construye una matriz $X \in \mathbb{R}^{V \times V}$ donde cada entrada $X_{ij}$ representa el número de veces que la palabra $j$ aparece en el contexto de la palabra $i$.

### Objetivo del modelo

El objetivo es encontrar vectores $\mathbf{w}_i$ y $\tilde{\mathbf{w}}_j$ tales que:

$$
\mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j = \log X_{ij}
$$

Donde $b_i$ y $\tilde{b}_j$ son términos de sesgo.

### Función de costo

Se define una función de costo ponderada de mínimos cuadrados:

$$
J = \sum_{i,j=1}^{V} f(X_{ij}) \left( \mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j - \log X_{ij} \right)^2
$$

Donde la función de ponderación $f(X_{ij})$ es:

$$
f(X_{ij}) = \begin{cases}
\left( \frac{X_{ij}}{X_{\text{max}}} \right)^\alpha & \text{si } X_{ij} < X_{\text{max}} \\
1 & \text{si } X_{ij} \geq X_{\text{max}}
\end{cases}
$$

Con típicamente $X_{\text{max}} = 100$ y $\alpha = 0.75$.

### Interpretación

- El modelo busca capturar relaciones semánticas mediante la aproximación de las razones de probabilidades de co-ocurrencia.
- La utilización de la logaritmo de $X_{ij}$ permite modelar relaciones lineales en el espacio vectorial.


## Algoritmos de los métodos

### Algoritmo de Brown clustering

1. **Inicialización**: Asignar cada palabra a su propio cluster.

2. **Cálculo de probabilidades**: Calcular $P(c)$ y $P(c_i, c_j)$.

3. **Búsqueda de fusión óptima**: Para cada par de clusters $(c_i, c_j)$, calcular la disminución en la información mutua $\Delta I(c_i, c_j)$ si se fusionan.

4. **Fusión de clusters**: Fusionar el par $(c_i, c_j)$ que minimiza $\Delta I$.

5. **Actualización**: Actualizar las probabilidades y repetir desde el paso 3 hasta alcanzar el número deseado de clusters.

### Algoritmo de LSA

1. **Construcción de la matriz**: Crear la matriz término-documento $X$ con pesos apropiados.

2. **Aplicación de SVD**: Descomponer $X$ en $U \Sigma V^\top$.

3. **Selección de dimensiones**: Elegir el número $k$ de dimensiones significativas.

4. **Proyección**: Obtener las representaciones reducidas $U_k \Sigma_k$ y $V_k \Sigma_k$ para términos y documentos.

### Algoritmo de Word2vec (Skip-Gram con negative sampling)

1. **Inicialización**: Asignar vectores aleatorios a las palabras.

2. **Recorrido del corpus**: Para cada palabra $w_t$ en el corpus:

   a. **Selección del contexto**: Identificar palabras de contexto $w_{t+j}$ dentro de la ventana.

   b. **Actualización de parámetros**: Para cada palabra de contexto:

      - Calcular la pérdida con negative sampling.
      - Actualizar los vectores $\mathbf{w}_{w_t}$ y $\mathbf{w}'_{w_{t+j}}$.

3. **Iteración**: Repetir el paso 2 para varias épocas hasta la convergencia.

### Algoritmo de GloVe

1. **Construcción de la matriz**: Calcular la matriz de co-ocurrencia $X$.

2. **Inicialización**: Asignar vectores y sesgos aleatorios a las palabras.

3. **Optimización**: Minimizar la función de costo $J$ utilizando un método de gradiente (e.g., AdaGrad):

   a. **Cálculo del gradiente**: Para cada par $(i, j)$, calcular el gradiente de $J$ respecto a $\mathbf{w}_i$, $\tilde{\mathbf{w}}_j$, $b_i$, $\tilde{b}_j$.

   b. **Actualización de parámetros**: Actualizar los vectores y sesgos.

4. **Iteración**: Repetir hasta la convergencia.

### Ejercicio
*
Implementa todas las técnicas incluyendo Brown Clustering, latent semantic analysis (LSA), Word2vec (CBOW y Skip-Gram con negative sampling). Además, analizar y optimizar la complejidad algorítmica de los métodos desarrollados.

#### **Descripción del ejercicio**

1. **Preparación del entorno de trabajo**
   - **Lenguaje de programación:** Python 3.x
   - **Requisitos:** Utilizar únicamente las librerías estándar de Python. No se permite el uso de librerías externas `gensim` o `spaCy`, etc.
   - **Herramientas sugeridas:**
     - **Estructuras de Datos:** Listas, diccionarios, conjuntos.
     - **Manejo de archivos:** Lectura y escritura de archivos de texto.
     - **Optimización:** Uso eficiente de estructuras de datos y algoritmos para manejar grandes volúmenes de datos.

2. **Selección y preparación del corpus**
   - **Corpus Sugerido:** [Corpus de Wikipedia en Español](https://dumps.wikimedia.org/eswiki/latest/eswiki-latest-pages-articles.xml.bz2) o cualquier otro corpus extenso y representativo en español.
   - **Preprocesamiento: (2 puntos)**
     - **Descarga y extracción:** Descargar el dump de Wikipedia y extraer el texto relevante.
     - **Tokenización (0.5 puntos):** Implementar un tokenizer desde cero que divida el texto en palabras, manejando puntuación, mayúsculas/minúsculas y otros aspectos lingüísticos básicos.
     - **Lematización/stemming (0.5 puntos):** Crear una función simple de stemming o lematización para reducir las palabras a su forma base. (Nota: Dado que no se usan librerías externas, la implementación será básica y posiblemente menos precisa).
     - **Remoción de stopwords (0.5 puntos):** Definir una lista de palabras vacías (stopwords) en español y eliminar todas las ocurrencias de estas palabras en el corpus.
     - **Filtrado de palabras raras (0.5 puntos):** Establecer un umbral de frecuencia (e.g., eliminar palabras que aparecen menos de 5 veces) para reducir el tamaño del vocabulario.

3. **Implementación de técnicas**

   ##### **A. Brown clustering (4 puntos)**
   - **Descripción:** Implementar el algoritmo de Brown Clustering para agrupar palabras basándose en su contexto.
   - **Pasos:**
     1. **Inicialización (0.5 puntos):** Asignar cada palabra a su propio cluster utilizando estructuras de datos como diccionarios para mapear palabras a clusters.
     2. **Cálculo de probabilidades (0.5 puntos):** Implementar funciones para estimar las probabilidades $P(c)$ y $P(c_i, c_j)$ a partir del corpus procesado.
     3. **Búsqueda de fusión óptima (0.5 puntos):** Desarrollar una función que evalúe, para cada par de clusters, la disminución en la información mutua $\Delta I(c_i, c_j)$ si se fusionan.
     4. **Fusión de clusters (0.5 puntos):** Implementar la lógica para unir el par de clusters que minimiza $\Delta I$.
     5. **Repetición:** Continuar el proceso hasta alcanzar el número deseado de clusters (e.g., 100 clusters).
   - **Mejoras en complejidad :**
     - **Estructuras de datos eficientes (1 punto):** Utilizar estructuras como árboles binarios o matrices de adyacencia para manejar pares de clusters y sus probabilidades.
     - **Técnicas de poda (1 punto):** Implementar métodos para limitar el número de fusiones candidatas en cada iteración, reduciendo así el tiempo de cómputo.

   ##### **B. Latent semantic analysis (LSA) (3 puntos)**
   - **Descripción:** Aplicar LSA para reducir la dimensionalidad y capturar relaciones semánticas.
   - **Pasos:**
     1. **Construcción de la matriz término-socumento:** Crear una matriz $X$ donde las filas representen términos y las columnas representen documentos, asignando pesos como TF-IDF.
     2. **Implementación de SVD (1 punto):** Desarrollar una función desde cero para realizar la descomposición en valores singulares (SVD) de la matriz $X$. Dado que SVD es complejo de implementar, considerar simplificaciones o métodos iterativos básicos.
     3. **Reducción de Dimensionalidad (0.5 puntos):** Seleccionar los $k$ valores singulares más grandes para obtener $X_k = U_k \Sigma_k V_k^\top$.
     4. **Proyección (0.5 puntos):** Representar términos y documentos en el espacio reducido utilizando las matrices resultantes.
   - **Mejoras en complejidad:**
     - **Métodos de SVD eficientes (opcional):** Implementar algoritmos iterativos como el método de potencia para calcular los valores y vectores singulares más significativos.
     - **Manejo de matrices dispersas (1 punto):** Optimizar el almacenamiento y operaciones sobre matrices dispersas para reducir el uso de memoria y tiempo de procesamiento.

   ##### **C. Word2vec (2 puntos)**
   - **Descripción:** Entrenar modelos Word2vec utilizando las arquitecturas CBOW y Skip-Gram con Negative Sampling.
   - **Pasos:**
     1. **Preprocesamiento:** Realizar los mismos pasos de preprocesamiento que en la preparación del corpus.
     2. **Implementación CBOW (1 punto):**
        - **Ventana de contexto:** Definir una ventana de tamaño $c$ alrededor de la palabra objetivo.
        - **Función de pérdida:** Implementar la función de pérdida negativa logarítmica para predecir la palabra objetivo basada en el contexto.
        - **Optimización:** Desarrollar un algoritmo de descenso de gradiente básico para actualizar los vectores de palabras.
     3. **Implementación skip-Gram con negative sampling (1 punto):**
        - **Predicción de contexto:** Para cada palabra objetivo, predecir palabras de contexto dentro de la ventana.
        - **Negative sampling:** Implementar una función para muestrear palabras negativas de manera eficiente.
        - **Función de pérdida:** Implementar la función de pérdida que considera tanto los pares positivos como los negativos.
   - **Mejoras en complejidad:**
     - **Optimización Paralela (opcional):** Simular paralelización utilizando múltiples hilos o procesos para acelerar el entrenamiento.
     - **Tablas de muestreo eficientes (opcional):** Implementar estructuras de datos como tablas de alias para realizar el muestreo negativo de manera más rápida.

   ##### **D. GloVe (Global vectors for word representation) (2 puntos)**
   - **Descripción:** Entrenar el modelo GloVe para obtener embeddings de palabras basados en co-ocurrencias globales.
   - **Pasos:**
     1. **Construcción de la Matriz de co-ocurrencia $X$:** Implementar una función que recorra el corpus y cuente las co-ocurrencias de palabras dentro de una ventana determinada.
     2. **Inicialización de vectores y sesgos:** Asignar vectores y términos de sesgo iniciales a cada palabra de manera aleatoria.
     3. **Optimización de la función de costo (1 punto):** Desarrollar una función para minimizar $J = \sum_{i,j=1}^{V} f(X_{ij}) (\mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j - \log X_{ij})^2$ utilizando un método de gradiente básico como el descenso de gradiente estocástico.
     4. **Iteración hasta convergencia:** Repetir el proceso de optimización hasta que la función de costo converja o se alcance un número máximo de iteraciones.
   - **Mejoras en complejidad:**
     - **Paralelización de actualizaciones (opcional):** Implementar mecanismos para actualizar múltiples vectores simultáneamente sin conflictos.
     - **Optimización de funciones de ponderación (1 punto):** Simplificar o precomputar partes de la función de ponderación $f(X_{ij})$ para acelerar el cálculo durante la optimización.

4. **Evaluación y comparación ( 3 puntos)**
   - **Tareas de evaluación:**
     - **Analogías semánticas (0.5 puntos):** Evaluar la capacidad de los embeddings para resolver analogías simples (e.g., "rey" es a "reina" como "hombre" es a "mujer").
     - **Clasificación de textos (0.5 puntos):** Utilizar las representaciones obtenidas para clasificar documentos en categorías predefinidas mediante implementaciones básicas de algoritmos de clasificación (e.g., k-NN, Naive Bayes).
     - **Clustering de palabras (0.5 puntos):** Analizar la coherencia de los clusters obtenidos por Brown Clustering comparados con los embeddings de Word2vec y GloVe implementados.
   - **Métricas:**
     - **Precisión y Recall (0.5 puntos):** Implementar métricas básicas para evaluar la exactitud de las tareas de clasificación y analogías.
     - **Coeficiente de Silhouette (0.5 puntos):** Desarrollar una función para calcular el coeficiente de Silhouette y evaluar la calidad del clustering.
     - **Tiempo de ejecución y uso de memoria (0.5 puntos):** Medir y comparar los tiempos de ejecución y el uso de memoria de las implementaciones, especialmente después de aplicar mejoras en la complejidad algorítmica.


Debes tener en cuenta que cuando implementes tus algoritmos las mejoras se deben dar de la siguiente forma:

   - **Brown Clustering:**
     - **Complejidad inicial:** Aproximadamente $O(N \cdot |V|^2)$, donde $N$ es el número de bigramas y $|V|$ el tamaño del vocabulario.
     - **Mejoras implementadas:** Uso de estructuras de datos eficientes para reducir la complejidad a $O(N \cdot |V| \log |V|)$ mediante técnicas de poda y estructuras jerárquicas.
   - **LSA:**
     - **Complejidad inicial:** $O(mn \min(m, n))$ para una implementación básica de SVD.
     - **Mejoras implementadas:** Implementación de métodos iterativos para Truncated SVD, reduciendo la complejidad a $O(mnk)$, donde $k$ es la dimensión reducida.
   - **Word2vec y GloVe:**
     - **Complejidad inicial:** $O(T \cdot c \cdot V)$, donde $T$ es el tamaño del corpus y $c$ la ventana de contexto.
     - **Mejoras implementadas:** Implementación de negative sampling para reducir la complejidad a $O(T \cdot c \cdot K)$, donde $K$ es el número de muestras negativas, y optimizaciones en la actualización de vectores para disminuir tiempos de cómputo.




In [ ]:
import re
import math
import heapq
import json
from collections import Counter, defaultdict
from typing import List, Tuple, Dict, Set

def tokenize(text: str) -> List[str]:
    """
    Tokeniza el texto en palabras, eliminando puntuación y convirtiendo a minúsculas.
    Maneja caracteres especiales comunes en español.
    """
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    text = re.sub(r'[^a-záéíóúüñ0-9]+', ' ', text)
    # Dividir el texto en tokens
    tokens = text.split()
    return tokens

def simple_stemmer(word: str) -> str:
    """
    Aplica reglas básicas de stemming en español.
    Implementación más robusta con reglas adicionales.
    """
    suffixes = [
        'amiento', 'amientos', 'ación', 'aciones', 'ador', 'adores', 'adora', 'adoras',
        'ablemente', 'abilidades', 'idades', 'idad', 'able', 'ables', 'ible', 'ibles',
        'ación', 'aciones', 'ción', 'ciones', 'anza', 'anzas', 'mente',
        'ando', 'iendo', 'ado', 'ada', 'idos', 'idas', 'ar', 'er', 'ir', 'es',
        'os', 'as', 'a', 'o', 'e', 'í', 'ó'
    ]
    for suffix in sorted(suffixes, key=lambda x: -len(x)):
        if word.endswith(suffix) and len(word) > len(suffix) + 2:
            return word[:-len(suffix)]
    return word

def stem_tokens(tokens: List[str]) -> List[str]:
    """
    Aplica stemming a una lista de tokens.
    """
    return [simple_stemmer(token) for token in tokens]

def load_stopwords(file_path: str) -> Set[str]:
    """
    Carga una lista de stopwords desde un archivo de texto.
    Cada línea del archivo debe contener una stopword.
    """
    stopwords = set()
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                word = line.strip().lower()
                if word:
                    stopwords.add(word)
    except FileNotFoundError:
        print(f"Archivo de stopwords no encontrado en {file_path}. Asegúrate de crearlo.")
    return stopwords

def remove_stopwords(tokens: List[str], stopwords: Set[str]) -> List[str]:
    """
    Elimina las stopwords de una lista de tokens.
    """
    return [token for token in tokens if token not in stopwords]

def filter_rare_words(tokens: List[str], threshold: int = 2) -> Tuple[List[str], Counter]:
    """
    Filtra palabras que aparecen menos de `threshold` veces.
    Retorna la lista de tokens filtrados y el conteo de palabras.
    """
    freq = Counter(tokens)
    return [token for token in tokens if freq[token] >= threshold], freq

def filter_rare_words_dynamic(tokens: List[str], percentile: float = 0.1) -> Tuple[List[str], Counter]:
    """
    Filtra palabras basándose en un percentil de frecuencia.
    Las palabras con frecuencia por debajo del percentil especificado son eliminadas.
    """
    freq = Counter(tokens)
    frequencies = list(freq.values())
    frequencies.sort()
    cutoff_index = int(len(frequencies) * percentile)
    if cutoff_index < 1:
        cutoff_index = 1
    cutoff = frequencies[cutoff_index]
    return [token for token in tokens if freq[token] >= cutoff], freq

def generate_bigrams(tokens: List[str]) -> List[Tuple[str, str]]:
    """
    Genera bigramas a partir de una lista de tokens.
    """
    return list(zip(tokens[:-1], tokens[1:]))

def initialize_clusters(vocab: List[str]) -> Dict[str, int]:
    """
    Inicializa cada palabra en su propio cluster.
    Retorna un diccionario mapeando palabras a identificadores de clusters.
    """
    clusters = {word: idx for idx, word in enumerate(vocab)}
    return clusters

def calculate_probabilities(
    clusters: Dict[str, int],
    word_counts: Counter,
    bigram_counts: Counter,
    total_words: int,
    total_bigrams: int
) -> Tuple[Dict[int, float], Dict[Tuple[int, int], float]]:
    """
    Calcula las probabilidades P(c) y P(c_i, c_j).
    """
    P_c = defaultdict(float)
    for word, cluster in clusters.items():
        P_c[cluster] += word_counts[word] / total_words

    P_ci_cj = defaultdict(float)
    for (w_i, w_j), freq in bigram_counts.items():
        c_i = clusters[w_i]
        c_j = clusters[w_j]
        P_ci_cj[(c_i, c_j)] += freq / total_bigrams

    return dict(P_c), dict(P_ci_cj)

def compute_mutual_information(
    P_c: Dict[int, float],
    P_ci_cj: Dict[Tuple[int, int], float]
) -> float:
    """
    Calcula la información mutua I(C).
    """
    I = 0.0
    for (c_i, c_j), P_val in P_ci_cj.items():
        if P_val > 0 and P_c[c_i] > 0 and P_c[c_j] > 0:
            I += P_val * math.log(P_val / (P_c[c_i] * P_c[c_j]), 2)
    return I

def find_best_merge(
    clusters: Dict[str, int],
    P_c: Dict[int, float],
    P_ci_cj: Dict[Tuple[int, int], float],
    word_counts: Counter,
    bigram_counts: Counter,
    total_bigrams: int
) -> Tuple[Tuple[int, int], float]:
    """
    Encuentra el mejor par de clusters para fusionar que maximiza la información mutua.
    Utiliza una cola de prioridad para seleccionar el mejor par de manera eficiente.
    """
    # Usamos una heap como cola de prioridad
    heap = []
    # Pre-calculamos la información mutua actual
    current_I = compute_mutual_information(P_c, P_ci_cj)
    
    unique_clusters = list(P_c.keys())
    n = len(unique_clusters)
    
    for i in range(n):
        for j in range(i + 1, n):
            c_i = unique_clusters[i]
            c_j = unique_clusters[j]
            
            # Calcular el nuevo P(c) para la fusión
            P_c_new = P_c[c_i] + P_c[c_j]
            
            # Calcular la nueva P(ci, cj) después de la fusión
            P_ci_cj_new = defaultdict(float)
            for (a, b), P_val in P_ci_cj.items():
                # Actualizar los clusters después de la fusión
                a_new = c_i if a == c_i or a == c_j else a
                b_new = c_j if b == c_i or b == c_j else b
                key = tuple(sorted((a_new, b_new)))
                P_ci_cj_new[key] += P_val
            
            # Calcular la nueva información mutua
            I_new = compute_mutual_information(P_c, P_ci_cj_new)
            
            # Calcular el delta de información mutua
            delta_I = I_new - current_I
            
            # Usamos -delta_I porque heapq es una min-heap y queremos una max-heap
            heapq.heappush(heap, (-delta_I, (c_i, c_j)))
    
    if not heap:
        return None, 0.0
    
    # Obtener la fusión con el mayor delta_I
    best_delta_I, best_pair = heapq.heappop(heap)
    return best_pair, -best_delta_I

def save_clusters(clusters: Dict[str, int], file_path: str) -> None:
    """
    Guarda los clusters en un archivo JSON.
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(clusters, f, ensure_ascii=False, indent=4)

def load_clusters(file_path: str) -> Dict[str, int]:
    """
    Carga los clusters desde un archivo JSON.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        clusters = json.load(f)
    # Convertir las claves de los clusters a enteros si es necesario
    return {word: int(cluster) for word, cluster in clusters.items()}

def print_progress(current: int, total: int, bar_length: int = 50) -> None:
    """
    Imprime una barra de progreso en la consola.
    """
    fraction = current / total
    arrow = '=' * int(fraction * bar_length)
    padding = ' ' * (bar_length - len(arrow))
    ending = '\n' if current == total else '\r'
    print(f'Progreso: [{arrow}{padding}] {int(fraction * 100)}%', end=ending)

def evaluate_clusters(
    clusters: Dict[str, int],
    word_counts: Counter,
    bigram_counts: Counter
) -> float:
    """
    Evalúa la calidad de los clusters calculando la entropía de las clases.
    Menor entropía indica mayor coherencia dentro de las clases.
    """
    cluster_sizes = defaultdict(int)
    for word, cluster in clusters.items():
        cluster_sizes[cluster] += word_counts[word]
    
    total_words = sum(cluster_sizes.values())
    entropy = 0.0
    for cluster, size in cluster_sizes.items():
        p = size / total_words
        if p > 0:
            entropy -= p * math.log(p, 2)
    return entropy

def brown_clustering_main(
    corpus_text: str,
    stopwords_file: str,
    desired_clusters: int = 100,
    rare_word_threshold: int = 2,
    save_clusters_path: str = 'clusters.json',
    load_clusters_path: str = None
) -> Dict[str, int]:
    """
    Ejecuta el algoritmo de Brown Clustering con mejoras avanzadas.
    
    Parámetros:
    - corpus_text: Texto del corpus a procesar.
    - stopwords_file: Ruta al archivo de stopwords.
    - desired_clusters: Número de clusters deseados.
    - rare_word_threshold: Umbral de frecuencia para filtrar palabras raras.
    - save_clusters_path: Ruta para guardar los clusters resultantes.
    - load_clusters_path: Ruta para cargar clusters existentes (opcional).
    
    Retorna:
    - Diccionario de palabras mapeadas a sus clusters.
    """
    # Tokenizar el texto
    tokens = tokenize(corpus_text)
    print(f"Total de tokens después de tokenización: {len(tokens)}")
    
    # Aplicar stemming
    stemmed_tokens = stem_tokens(tokens)
    print(f"Total de tokens después de stemming: {len(stemmed_tokens)}")
    
    # Cargar stopwords y eliminar
    stopwords = load_stopwords(stopwords_file)
    if not stopwords:
        print("No se cargaron stopwords. Asegúrate de que el archivo exista y tenga contenido.")
    filtered_tokens = remove_stopwords(stemmed_tokens, stopwords)
    print(f"Total de tokens después de eliminar stopwords: {len(filtered_tokens)}")
    
    # Filtrar palabras raras
    filtered_tokens, word_freq = filter_rare_words(filtered_tokens, threshold=rare_word_threshold)
    print(f"Total de tokens después de filtrar palabras raras (umbral={rare_word_threshold}): {len(filtered_tokens)}")
    print(f"Vocabulario tamaño: {len(word_freq)}")
    
    # Generar bigrams
    bigrams = generate_bigrams(filtered_tokens)
    print(f"Total de bigrams generados: {len(bigrams)}")
    
    # Contar frecuencias
    word_counts, bigram_counts, total_words, total_bigrams = count_frequencies(filtered_tokens, bigrams)
    print(f"Total de palabras contadas: {total_words}")
    print(f"Total de bigrams contados: {total_bigrams}")
    
    # Inicializar clusters
    vocab = list(word_freq.keys())
    clusters = initialize_clusters(vocab)
    print(f"Clusters inicializados: {len(clusters)} clusters.")
    
    # Si se proporciona un archivo de clusters existentes, cargarlo
    if load_clusters_path:
        clusters = load_clusters(load_clusters_path)
        print(f"Clusters cargados desde {load_clusters_path}. Total de clusters: {len(set(clusters.values()))}")
    
    # Ejecutar Brown Clustering
    current_iteration = 0
    max_iterations = len(set(clusters.values())) - desired_clusters
    print(f"Iniciando Brown Clustering: {len(set(clusters.values()))} clusters iniciales, objetivo {desired_clusters} clusters.")
    while len(set(clusters.values())) > desired_clusters and current_iteration < max_iterations:
        P_c, P_ci_cj = calculate_probabilities(clusters, word_counts, bigram_counts, total_words, total_bigrams)
        best_pair, best_delta_I = find_best_merge(clusters, P_c, P_ci_cj, word_counts, bigram_counts, total_bigrams)
        if best_pair is None:
            print("No se encontraron más fusiones posibles.")
            break
        c_i, c_j = best_pair
        new_cluster_id = max(P_c.keys()) + 1
        clusters = merge_clusters(clusters, c_i, c_j, new_cluster_id)
        current_iteration += 1
        # Mostrar progreso
        print_progress(current_iteration, max_iterations)
    
    # Guardar clusters resultantes
    save_clusters(clusters, save_clusters_path)
    print(f"\nClusters guardados en {save_clusters_path}")
    
    # Evaluar clusters
    entropy = evaluate_clusters(clusters, word_counts, bigram_counts)
    print(f"Entropía de los clusters: {entropy:.4f}")
    
    return clusters

def count_frequencies(tokens: List[str], bigrams: List[Tuple[str, str]]) -> Tuple[Counter, Counter, int, int]:
    """
    Cuenta las frecuencias de las palabras y las bigramas.
    """
    word_counts = Counter(tokens)
    bigram_counts = Counter(bigrams)
    total_words = sum(word_counts.values())
    total_bigrams = sum(bigram_counts.values())
    return word_counts, bigram_counts, total_words, total_bigrams

def merge_clusters(clusters: Dict[str, int], c_i: int, c_j: int, new_cluster_id: int) -> Dict[str, int]:
    """
    Fusiona dos clusters en un nuevo cluster.
    """
    for word, cluster in clusters.items():
        if cluster == c_i or cluster == c_j:
            clusters[word] = new_cluster_id
    return clusters

# Ejemplo de Uso Completo
if __name__ == "__main__":
    # Ejemplo de corpus de texto
    corpus_text = """
    El rápido zorro marrón salta sobre el perro perezoso. 
    La vida es bella y llena de oportunidades. 
    Python es un lenguaje de programación poderoso y versátil. 
    El aprendizaje automático es una rama de la inteligencia artificial. 
    Aprender a programar en Python facilita el desarrollo de aplicaciones complejas.
    La inteligencia artificial y el aprendizaje automático están revolucionando la tecnología.
    """
    
    # Crear un archivo de stopwords si no existe
    stopwords_file = 'stopwords_es.txt'
    try:
        with open(stopwords_file, 'r', encoding='utf-8') as f:
            pass  # El archivo existe
    except FileNotFoundError:
        # Crear el archivo con algunas stopwords básicas
        with open(stopwords_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join([
                'el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas', 'y', 'de', 'en',
                'es', 'con', 'por', 'para', 'a', 'e', 'o', 'u', 'del', 'al', 'si',
                'que', 'se', 's'
            ]))
        print(f"Archivo de stopwords creado en {stopwords_file}")
    
    # Ejecutar Brown Clustering
    clusters_finales = brown_clustering_main(
        corpus_text=corpus_text,
        stopwords_file=stopwords_file,
        desired_clusters=5,
        rare_word_threshold=1,
        save_clusters_path='clusters_finales.json'
    )
    
    # Mostrar los clusters finales
    print("\nClusters finales:")
    for word, cluster in clusters_finales.items():
        print(f"Palabra: {word}, Cluster: {cluster}")


Esta es una implementación funcional del algoritmo de Brown Clustering. Es adecuado para pequeños conjuntos de datos y lo que pide el problema. Para aplicaciones más serias y corpora extensos, se recomienda implementar optimizaciones adicionales y posiblemente utilizar librerías especializadas para mejorar el rendimiento y la precisión.

Combinemos primero con LSA y Brown Clustering.

In [ ]:
import re
import math
import random
from collections import Counter, defaultdict
import copy

# 1. Preprocesamiento del corpus

def leer_corpus(ruta_archivo):
    """
    Lee el corpus desde un archivo de texto.
    Supone que cada línea es un documento.
    """
    documentos = []
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            for linea in archivo:
                if linea.strip():  # Ignorar líneas vacías
                    documentos.append(linea.strip())
    except FileNotFoundError:
        print(f"Error: El archivo '{ruta_archivo}' no se encontró.")
    return documentos

def tokenizer(texto):
    """
    Tokeniza el texto en palabras, eliminando puntuación y convirtiendo a minúsculas.
    """
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúñü]+', ' ', texto)
    tokens = texto.split()
    return tokens

def stemming(palabra):
    """
    Aplica una versión simplificada de stemming para palabras en español.
    """
    sufijos = ['ción', 'sión', 'mente', 'ando', 'iendo', 'ado', 'ido', 'ar', 'er', 'ir',
               'es', 'os', 'as', 'e', 'a', 'o']
    for sufijo in sufijos:
        if palabra.endswith(sufijo) and len(palabra) > len(sufijo) + 2:
            return palabra[:-len(sufijo)]
    return palabra

def aplicar_stemming(tokens):
    """
    Aplica stemming a una lista de tokens.
    """
    return [stemming(token) for token in tokens]

# Lista de stopwords en español (parcial)
STOPWORDS = {
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las',
    'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como',
    'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta',
    'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta',
    'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos',
    'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos',
    'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro',
    'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes',
    'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas',
    'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus',
    'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía',
    'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya',
    'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras',
    'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy',
    'estás', 'está', 'estamos', 'estáis', 'están', 'esté', 'estés',
    'estemos', 'estéis', 'estén', 'estaré', 'estarás', 'estará',
    'estaremos', 'estaréis', 'estarán', 'estaría', 'estarías',
    'estaríamos', 'estaríais', 'estarían', 'estaba', 'estabas',
    'estábamos', 'estabais', 'estaban', 'estuve', 'estuviste',
    'estuvo', 'estuvimos', 'estuvisteis', 'estuvieron', 'estuviera',
    'estuvieras', 'estuviéramos', 'estuvierais', 'estuvieran',
    'estuviese', 'estuvieses', 'estuviésemos', 'estuvieseis',
    'estuviesen', 'estando', 'estado', 'estada', 'estados', 'estadas',
    'estad'
}

def remover_stopwords(tokens):
    """
    Remueve las stopwords de una lista de tokens.
    """
    return [token for token in tokens if token not in STOPWORDS]

def filtrar_palabras_raras(tokens, umbral=5):
    """
    Filtra las palabras que aparecen menos de 'umbral' veces.
    """
    contador = Counter(tokens)
    vocabulario = {palabra for palabra, freq in contador.items() if freq >= umbral}
    tokens_filtrados = [token for token in tokens if token in vocabulario]
    return tokens_filtrados, vocabulario

def preprocesar_documentos(documentos, umbral=5):
    """
    Preprocesa cada documento: tokenización, stemming, remoción de stopwords y filtrado.
    """
    documentos_tokens = []
    contador_total = Counter()
    for doc in documentos:
        tokens = tokenizer(doc)
        tokens = aplicar_stemming(tokens)
        tokens = remover_stopwords(tokens)
        documentos_tokens.append(tokens)
        contador_total.update(tokens)
    # Filtrar palabras raras
    vocabulario = {palabra for palabra, freq in contador_total.items() if freq >= umbral}
    if not vocabulario:
        print("Advertencia: El vocabulario está vacío después del filtrado. Considera reducir el umbral.")
    documentos_tokens_filtrados = [
        [token for token in tokens if token in vocabulario]
        for tokens in documentos_tokens
    ]
    return documentos_tokens_filtrados, vocabulario


# 2. Brown clustering (simplificado)

def inicializar_clusters(vocabulario):
    """
    Inicializa cada palabra en su propio clúster.
    """
    clusters = {palabra: idx for idx, palabra in enumerate(vocabulario)}
    return clusters

def calcular_probabilidades(tokens, clusters):
    """
    Calcula P(c) y P(c_i, c_j) a partir de los tokens y la asignación de clusters.
    """
    contador_clusters = Counter()
    contador_clusters_parejas = Counter()

    for i in range(len(tokens)-1):
        palabra = tokens[i]
        siguiente_palabra = tokens[i+1]
        cluster_actual = clusters.get(palabra, None)
        cluster_siguiente = clusters.get(siguiente_palabra, None)
        if cluster_actual is not None and cluster_siguiente is not None:
            contador_clusters[cluster_actual] += 1
            contador_clusters_parejas[(cluster_actual, cluster_siguiente)] += 1

    total_clusters = sum(contador_clusters.values())
    if total_clusters == 0:
        return {}, {}
    P_c = {c: count / total_clusters for c, count in contador_clusters.items()}

    total_parejas = sum(contador_clusters_parejas.values())
    if total_parejas == 0:
        return P_c, {}
    P_ci_cj = {pair: count / total_parejas for pair, count in contador_clusters_parejas.items()}

    return P_c, P_ci_cj

def informacion_mutua(P_c, P_ci_cj):
    """
    Calcula la información mutua I(c_i, c_j).
    """
    I = 0.0
    for (c_i, c_j), P in P_ci_cj.items():
        if P > 0 and c_i in P_c and c_j in P_c:
            I += P * math.log(P / (P_c[c_i] * P_c[c_j]) + 1e-10)
    return I

def buscar_fusion_optima(P_c, P_ci_cj, clusters, vocabulario, tokens):
    """
    Encuentra el par de clústeres cuya fusión minimiza la disminución de la información mutua.
    """
    min_delta_I = float('inf')
    par_optimo = None
    clusters_unicos = set(clusters.values())

    for c1 in clusters_unicos:
        for c2 in clusters_unicos:
            if c1 >= c2:
                continue
            # Simular fusión
            clusters_simulados = clusters.copy()
            nuevo_c = max(clusters_unicos) + 1
            for palabra in vocabulario:
                if clusters_simulados.get(palabra, None) == c1 or clusters_simulados.get(palabra, None) == c2:
                    clusters_simulados[palabra] = nuevo_c
            # Calcular nuevas probabilidades
            P_c_sim, P_ci_cj_sim = calcular_probabilidades(tokens, clusters_simulados)
            # Calcular diferencia en información mutua
            I_original = informacion_mutua(P_c, P_ci_cj)
            I_sim = informacion_mutua(P_c_sim, P_ci_cj_sim)
            delta_I = I_original - I_sim
            if delta_I < min_delta_I:
                min_delta_I = delta_I
                par_optimo = (c1, c2)
    return par_optimo, min_delta_I

def fusionar_clusters(clusters, par):
    """
    Fusiona dos clústeres dados en 'par' y actualiza la asignación de clusters.
    """
    if par is None:
        return clusters
    c1, c2 = par
    nuevo_c = max(clusters.values()) + 1
    for palabra, c in clusters.items():
        if c == c1 or c == c2:
            clusters[palabra] = nuevo_c
    return clusters

def brown_clustering(documentos_tokens, vocabulario, num_clusters=100):
    """
    Implementa una versión simplificada de Brown Clustering.
    """
    clusters = inicializar_clusters(vocabulario)
    iteracion = 0
    tokens_flat = [token for doc in documentos_tokens for token in doc]
    while len(set(clusters.values())) > num_clusters:
        iteracion += 1
        print(f"Iteración {iteracion}: Número de clústeres = {len(set(clusters.values()))}")
        P_c, P_ci_cj = calcular_probabilidades(tokens_flat, clusters)
        if not P_ci_cj:
            print("No hay pares de clústeres para fusionar.")
            break
        par_optimo, delta_I = buscar_fusion_optima(P_c, P_ci_cj, clusters, vocabulario, tokens_flat)
        if par_optimo is None:
            print("No se pudo encontrar un par óptimo para fusionar.")
            break
        clusters = fusionar_clusters(clusters, par_optimo)
        print(f"Fusión de clústeres {par_optimo} en uno nuevo. Delta I = {delta_I:.6f}")
    return clusters


# 3. Latent Semantic Analysis (LSA)


def calcular_tf(documentos_tokens):
    """
    Calcula el término frecuencia (TF) para cada documento.
    """
    tf = []
    for tokens in documentos_tokens:
        contador = Counter(tokens)
        tf_doc = {term: count for term, count in contador.items()}
        tf.append(tf_doc)
    return tf

def calcular_idf(documentos_tokens, vocabulario):
    """
    Calcula el inverso de la frecuencia de documentos (IDF) para cada término.
    """
    N = len(documentos_tokens)
    df = Counter()
    for tokens in documentos_tokens:
        tokens_unicos = set(tokens)
        for term in tokens_unicos:
            df[term] += 1
    idf = {}
    for term in vocabulario:
        df_term = df.get(term, 0)
        idf[term] = math.log((N + 1) / (df_term + 1)) + 1  # Evitar división por cero
    return idf

def construir_matriz_tf_idf(documentos_tokens, vocabulario):
    """
    Construye la matriz TF-IDF.
    Retorna una matriz dispersa representada como una lista de diccionarios por término.
    """
    tf = calcular_tf(documentos_tokens)
    idf = calcular_idf(documentos_tokens, vocabulario)
    term_to_index = {term: idx for idx, term in enumerate(sorted(vocabulario))}
    doc_to_index = {idx: idx for idx in range(len(documentos_tokens))}
    m = len(vocabulario)
    n = len(documentos_tokens)
    if m == 0 or n == 0:
        print("Advertencia: La matriz TF-IDF no puede ser construida debido a dimensiones inválidas.")
        return [], term_to_index, doc_to_index
    # Inicializar matriz con diccionarios
    X = [dict() for _ in range(m)]
    for j, tf_doc in enumerate(tf):
        for term, freq in tf_doc.items():
            if term in term_to_index:
                i = term_to_index[term]
                X[i][j] = freq * idf[term]
    return X, term_to_index, doc_to_index

def matriz_discreta_transpuesta(X):
    """
    Calcula la transpuesta de una matriz dispersa.
    """
    if not X:
        return []
    num_cols = max((max(fila.keys()) if fila else -1) for fila in X) + 1
    Xt = [dict() for _ in range(num_cols)]
    for i, fila in enumerate(X):
        for j, valor in fila.items():
            Xt[j][i] = valor
    return Xt

def multiplicar_matriz_discreta_vector(X, v):
    """
    Multiplica una matriz dispersa X con un vector v.
    X: lista de diccionarios (filas)
    v: lista
    """
    resultado = []
    for fila in X:
        suma = 0.0
        for j, valor in fila.items():
            if j < len(v):
                suma += valor * v[j]
        resultado.append(suma)
    return resultado

def vector_norm(v):
    """
    Calcula la norma Euclidiana de un vector.
    """
    return math.sqrt(sum([x**2 for x in v]))

def multiplicar_vector_matriz(Xt, v):
    """
    Multiplica una matriz transpuesta dispersa Xt con un vector v.
    Xt: lista de diccionarios (columnas de la original)
    v: lista
    """
    resultado = [0.0 for _ in range(len(Xt))]
    for j, columna in enumerate(Xt):
        for i, valor in columna.items():
            if i < len(v):
                resultado[j] += valor * v[i]
    return resultado

def obtener_principal_autovector(X, num_iter=100, tol=1e-6):
    """
    Calcula el principal autovector y autovalor de la matriz X^T X utilizando el método de potencia.
    X: lista de diccionarios (filas)
    """
    Xt = matriz_discreta_transpuesta(X)
    if not Xt:
        print("Advertencia: La matriz transpuesta está vacía.")
        return None, None
    n = len(Xt)
    if n == 0:
        print("Advertencia: El número de documentos es cero.")
        return None, None
    v = [random.random() for _ in range(n)]
    # Normalizar
    norm = vector_norm(v)
    if norm == 0:
        print("Advertencia: Vector inicial tiene norma cero.")
        return None, None
    v = [x / norm for x in v]
    for iteracion in range(num_iter):
        # v_new = X^T * (X * v)
        Xv = multiplicar_matriz_discreta_vector(X, v)
        v_new = multiplicar_vector_matriz(Xt, Xv)
        norm = vector_norm(v_new)
        if norm == 0:
            print("Advertencia: Vector resultante tiene norma cero durante la iteración.")
            return None, None
        v_new = [x / norm for x in v_new]
        # Verificar convergencia
        diff = sum([abs(x - y) for x, y in zip(v, v_new)])
        if diff < tol:
            print(f"Convergencia alcanzada en iteración {iteracion+1}")
            break
        v = v_new
    # Calcular autovalor
    Xv = multiplicar_matriz_discreta_vector(X, v)
    autovalor = sum([x * y for x, y in zip(v, Xv)])
    return autovalor, v

def reducir_dimensionalidad(U, Sigma, V, k):
    """
    Reduce la dimensionalidad a 'k' dimensiones.
    """
    U_k = U[:k]
    Sigma_k = Sigma[:k]
    V_k = V[:k]
    return U_k, Sigma_k, V_k

def proyectar(U_k, Sigma_k, V_k, k):
    """
    Proyecta la matriz original en el espacio reducido.
    """
    # En una implementación completa, se calcularían U_k, Sigma_k y V_k
    # Aquí, como simplificación, se considera solo el primer componente
    X_k = []
    for i in range(len(U_k)):
        fila = []
        for j in range(len(V_k)):
            fila.append(U_k[i] * Sigma_k * V_k[j])
        X_k.append(fila)
    return X_k


# 4. Ejecución completa


def main():
    # Ruta al archivo del corpus
    ruta_corpus = 'corpus.txt'  # Reemplazar con la ruta real
    
    # Para propósitos de prueba, si el archivo no existe, usar un corpus de ejemplo
    import os
    if not os.path.isfile(ruta_corpus):
        print(f"El archivo '{ruta_corpus}' no existe. Usando un corpus de ejemplo.")
        documentos = [
            "Este es el primer documento.",
            "Este documento es el segundo documento.",
            "Y este es el tercer documento.",
            "¿Está este documento funcionando correctamente?",
            "El documento de prueba es este."
        ]
    else:
        print("Leyendo el corpus...")
        documentos = leer_corpus(ruta_corpus)
        print(f"Total de documentos leídos: {len(documentos)}")
    
    if not documentos:
        print("Error: El corpus está vacío. Por favor, proporciona un corpus válido.")
        return
    
    print("Preprocesando documentos...")
    documentos_tokens, vocabulario = preprocesar_documentos(documentos, umbral=2)  # Reducido el umbral para prueba
    print(f"Vocabulario después de filtrado: {len(vocabulario)} términos")
    if not vocabulario:
        print("Error: El vocabulario está vacío después del preprocesamiento. Ajusta el umbral o revisa el corpus.")
        return
    

    # Brown Clustering

    print("\nIniciando Brown Clustering...")
    clusters_finales = brown_clustering(documentos_tokens, vocabulario, num_clusters=5)  # Reducido el número de clústeres para prueba
    print(f"Brown Clustering completado. Total de clústeres: {len(set(clusters_finales.values()))}")

    # Latent Semantic Analysis (LSA)

    print("\nConstruyendo la matriz TF-IDF...")
    X, term_to_index, doc_to_index = construir_matriz_tf_idf(documentos_tokens, vocabulario)
    if not X:
        print("Error: La matriz TF-IDF no se pudo construir.")
        return
    print(f"Matriz TF-IDF construida con dimensiones: {len(X)} términos x {len(documentos_tokens)} documentos")
    
    print("\nCalculando el principal autovector y autovalor usando el método de potencia...")
    autovalor, autovector = obtener_principal_autovector(X)
    if autovalor is None or autovector is None:
        print("Error: No se pudo calcular el principal autovector y autovalor.")
        return
    print(f"Principal Autovalor: {autovalor}")
    print(f"Principal Autovector (primeros 10 elementos): {autovector[:10]}")
    
    # Reducción de dimensionalidad a k=1 (simplificación)
    k = 1
    print(f"\nReduciendo la dimensionalidad a k={k}...")
    # En esta implementación simplificada, solo se ha calculado un componente
    # Por lo tanto, U_k y V_k corresponden al autovector y autovalor
    U_k = autovector  # Representa los documentos en el espacio reducido
    Sigma_k = autovalor
    V_k = autovector  # Representa los términos en el espacio reducido
    
    print("\nProyectando términos y documentos en el espacio reducido...")
    # Dado que es k=1, la proyección es una dimensión
    # Para una implementación completa, se necesitarían más componentes
    # Aquí, se muestra una simplificación
    # Por ejemplo, representar cada documento por su peso en el principal componente
    documentos_proyectados = []
    for j in range(len(documentos_tokens)):
        peso = 0.0
        for i, fila in enumerate(X):
            peso += fila.get(j, 0.0) * V_k[i]
        documentos_proyectados.append(peso)
    
    print(f"Documentos proyectados en {k} dimensión(es).")
    print(f"Primeros 10 documentos proyectados: {documentos_proyectados[:10]}")
    
    # Representación de términos en el espacio reducido
    terminos_proyectados = []
    for term, idx in term_to_index.items():
        peso = U_k[idx] * Sigma_k
        terminos_proyectados.append((term, peso))
    print(f"Primeros 10 términos proyectados: {terminos_proyectados[:10]}")

if __name__ == "__main__":
    main()


**LSA**

Se presenta código optimizado en Python que realiza LSA desde cero utilizando únicamente las librerías estándar siguiendo lo solicitado.

In [ ]:
import re
import math
import random
from collections import Counter, defaultdict
import copy
import os


# 1. Preprocesamiento del corpus


def leer_corpus(ruta_archivo):
    """
    Lee el corpus desde un archivo de texto.
    Supone que cada línea es un documento.
    """
    documentos = []
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            for linea in archivo:
                if linea.strip():  # Ignorar líneas vacías
                    documentos.append(linea.strip())
    except FileNotFoundError:
        print(f"Advertencia: El archivo '{ruta_archivo}' no se encontró. Usando un corpus de ejemplo.")
        documentos = [
            "Este es el primer documento.",
            "Este documento es el segundo documento.",
            "Y este es el tercer documento.",
            "¿Está este documento funcionando correctamente?",
            "El documento de prueba es este."
        ]
    return documentos

def tokenizer(texto):
    """
    Tokeniza el texto en palabras, eliminando puntuación y convirtiendo a minúsculas.
    """
    texto = texto.lower()
    # Reemplazar caracteres no alfabéticos por espacios
    texto = re.sub(r'[^a-záéíóúñü]+', ' ', texto)
    tokens = texto.split()
    return tokens

def stemming(palabra):
    """
    Aplica una versión simplificada de stemming para palabras en español.
    """
    sufijos = ['ción', 'sión', 'mente', 'ando', 'iendo', 'ado', 'ido', 'ar', 'er', 'ir',
               'es', 'os', 'as', 'e', 'a', 'o']
    for sufijo in sufijos:
        if palabra.endswith(sufijo) and len(palabra) > len(sufijo) + 2:
            return palabra[:-len(sufijo)]
    return palabra

def aplicar_stemming(tokens):
    """
    Aplica stemming a una lista de tokens.
    """
    return [stemming(token) for token in tokens]

# Lista de stopwords en español (parcial)
STOPWORDS = {
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las',
    'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como',
    'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta',
    'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta',
    'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos',
    'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos',
    'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro',
    'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes',
    'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas',
    'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus',
    'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía',
    'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya',
    'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras',
    'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy',
    'estás', 'está', 'estamos', 'estáis', 'están', 'esté', 'estés',
    'estemos', 'estéis', 'estén', 'estaré', 'estarás', 'estará',
    'estaremos', 'estaréis', 'estarán', 'estaría', 'estarías',
    'estaríamos', 'estaríais', 'estarían', 'estaba', 'estabas',
    'estábamos', 'estabais', 'estaban', 'estuve', 'estuviste',
    'estuvo', 'estuvimos', 'estuvisteis', 'estuvieron', 'estuviera',
    'estuvieras', 'estuviéramos', 'estuvierais', 'estuvieran',
    'estuviese', 'estuvieses', 'estuviésemos', 'estuvieseis',
    'estuviesen', 'estando', 'estado', 'estada', 'estados', 'estadas',
    'estad'
}

def remover_stopwords(tokens):
    """
    Remueve las stopwords de una lista de tokens.
    """
    return [token for token in tokens if token not in STOPWORDS]

def filtrar_palabras_raras(tokens, umbral=5):
    """
    Filtra las palabras que aparecen menos de 'umbral' veces.
    """
    contador = Counter(tokens)
    vocabulario = {palabra for palabra, freq in contador.items() if freq >= umbral}
    tokens_filtrados = [token for token in tokens if token in vocabulario]
    return tokens_filtrados, vocabulario

def preprocesar_documentos(documentos, umbral=5):
    """
    Preprocesa cada documento: tokenización, stemming, remoción de stopwords y filtrado.
    """
    documentos_tokens = []
    contador_total = Counter()
    for doc in documentos:
        tokens = tokenizer(doc)
        tokens = aplicar_stemming(tokens)
        tokens = remover_stopwords(tokens)
        documentos_tokens.append(tokens)
        contador_total.update(tokens)
    # Filtrar palabras raras
    vocabulario = {palabra for palabra, freq in contador_total.items() if freq >= umbral}
    if not vocabulario:
        print("Advertencia: El vocabulario está vacío después del filtrado. Considera reducir el umbral.")
    documentos_tokens_filtrados = [
        [token for token in tokens if token in vocabulario]
        for tokens in documentos_tokens
    ]
    return documentos_tokens_filtrados, vocabulario


# 2. Latent Semantic Analysis (LSA)

def calcular_tf(documentos_tokens):
    """
    Calcula el término frecuencia (TF) para cada documento.
    """
    tf = []
    for tokens in documentos_tokens:
        contador = Counter(tokens)
        tf_doc = {term: count for term, count in contador.items()}
        tf.append(tf_doc)
    return tf

def calcular_idf(documentos_tokens, vocabulario):
    """
    Calcula el inverso de la frecuencia de documentos (IDF) para cada término.
    """
    N = len(documentos_tokens)
    df = Counter()
    for tokens in documentos_tokens:
        tokens_unicos = set(tokens)
        for term in tokens_unicos:
            df[term] += 1
    idf = {}
    for term in vocabulario:
        df_term = df.get(term, 0)
        idf[term] = math.log((N + 1) / (df_term + 1)) + 1  # Evitar división por cero
    return idf

def construir_matriz_tf_idf(documentos_tokens, vocabulario):
    """
    Construye la matriz TF-IDF.
    Retorna una matriz dispersa representada como una lista de diccionarios por término.
    """
    tf = calcular_tf(documentos_tokens)
    idf = calcular_idf(documentos_tokens, vocabulario)
    term_to_index = {term: idx for idx, term in enumerate(sorted(vocabulario))}
    doc_to_index = {idx: idx for idx in range(len(documentos_tokens))}
    m = len(vocabulario)
    n = len(documentos_tokens)
    if m == 0 or n == 0:
        print("Advertencia: La matriz TF-IDF no puede ser construida debido a dimensiones inválidas.")
        return [], term_to_index, doc_to_index
    # Inicializar matriz con diccionarios
    X = [dict() for _ in range(m)]
    for j, tf_doc in enumerate(tf):
        for term, freq in tf_doc.items():
            if term in term_to_index:
                i = term_to_index[term]
                X[i][j] = freq * idf[term]
    return X, term_to_index, doc_to_index

def matriz_discreta_transpuesta(X):
    """
    Calcula la transpuesta de una matriz dispersa.
    """
    if not X:
        return []
    num_cols = max((max(fila.keys()) if fila else -1) for fila in X) + 1
    Xt = [dict() for _ in range(num_cols)]
    for i, fila in enumerate(X):
        for j, valor in fila.items():
            Xt[j][i] = valor
    return Xt

def multiplicar_matriz_discreta_vector(X, v):
    """
    Multiplica una matriz dispersa X con un vector v.
    X: lista de diccionarios (filas)
    v: lista
    """
    resultado = []
    for fila in X:
        suma = 0.0
        for j, valor in fila.items():
            if j < len(v):
                suma += valor * v[j]
        resultado.append(suma)
    return resultado

def vector_norm(v):
    """
    Calcula la norma Euclidiana de un vector.
    """
    return math.sqrt(sum([x**2 for x in v]))

def multiplicar_vector_matriz(Xt, v):
    """
    Multiplica una matriz transpuesta dispersa Xt con un vector v.
    Xt: lista de diccionarios (columnas de la original)
    v: lista
    """
    resultado = [0.0 for _ in range(len(Xt))]
    for j, columna in enumerate(Xt):
        for i, valor in columna.items():
            if i < len(v):
                resultado[j] += valor * v[i]
    return resultado

def obtener_principal_autovector(X, num_iter=100, tol=1e-6):
    """
    Calcula el principal autovector y autovalor de la matriz X^T X utilizando el método de potencia.
    X: lista de diccionarios (filas)
    """
    Xt = matriz_discreta_transpuesta(X)
    if not Xt:
        print("Advertencia: La matriz transpuesta está vacía.")
        return None, None
    n = len(Xt)
    if n == 0:
        print("Advertencia: El número de documentos es cero.")
        return None, None
    # Inicializar vector aleatorio
    random.seed(42)  # Para reproducibilidad
    v = [random.random() for _ in range(n)]
    # Normalizar
    norm = vector_norm(v)
    if norm == 0:
        print("Advertencia: Vector inicial tiene norma cero.")
        return None, None
    v = [x / norm for x in v]
    for iteracion in range(num_iter):
        # v_new = X^T * (X * v)
        Xv = multiplicar_matriz_discreta_vector(X, v)
        v_new = multiplicar_vector_matriz(Xt, Xv)
        norm = vector_norm(v_new)
        if norm == 0:
            print("Advertencia: Vector resultante tiene norma cero durante la iteración.")
            return None, None
        v_new = [x / norm for x in v_new]
        # Verificar convergencia
        diff = sum([abs(x - y) for x, y in zip(v, v_new)])
        if diff < tol:
            print(f"Convergencia alcanzada en iteración {iteracion+1}")
            break
        v = v_new
    else:
        print("Advertencia: El método de potencia no ha convergido dentro del número máximo de iteraciones.")
    # Calcular autovalor
    Xv = multiplicar_matriz_discreta_vector(X, v)
    autovalor = sum([x * y for x, y in zip(v, Xv)])
    return autovalor, v

def reducir_dimensionalidad(U, Sigma, V, k):
    """
    Reduce la dimensionalidad a 'k' dimensiones.
    """
    U_k = U[:k]
    Sigma_k = Sigma[:k]
    V_k = V[:k]
    return U_k, Sigma_k, V_k

def proyectar(U_k, Sigma_k, V_k, k):
    """
    Proyecta la matriz original en el espacio reducido.
    """
    # En una implementación completa, se calcularían U_k, Sigma_k y V_k
    # Aquí, como simplificación, se considera solo el primer componente
    X_k = []
    for i in range(len(U_k)):
        fila = []
        for j in range(len(V_k)):
            fila.append(U_k[i] * Sigma_k * V_k[j])
        X_k.append(fila)
    return X_k


# 3. Ejecución completa de LSA

def main():
    # Ruta al archivo del corpus
    ruta_corpus = 'corpus1.txt'  # Reemplazar con la ruta real
    
    # Leer el corpus
    documentos = leer_corpus(ruta_corpus)
    print(f"Total de documentos leídos: {len(documentos)}")
    
    if not documentos:
        print("Error: El corpus está vacío. Por favor, proporciona un corpus válido.")
        return
    
    # Preprocesar documentos
    print("Preprocesando documentos...")
    documentos_tokens, vocabulario = preprocesar_documentos(documentos, umbral=2)  # Umbral reducido para prueba
    print(f"Vocabulario después de filtrado: {len(vocabulario)} términos")
    
    if not vocabulario:
        print("Error: El vocabulario está vacío después del preprocesamiento. Ajusta el umbral o revisa el corpus.")
        return
    
    # Construcción de la matriz TF-IDF
    print("\nConstruyendo la matriz TF-IDF...")
    X, term_to_index, doc_to_index = construir_matriz_tf_idf(documentos_tokens, vocabulario)
    if not X:
        print("Error: La matriz TF-IDF no se pudo construir.")
        return
    print(f"Matriz TF-IDF construida con dimensiones: {len(X)} términos x {len(documentos_tokens)} documentos")
    
    # Implementación de SVD (Método de Potencia)
    print("\nCalculando el principal autovector y autovalor usando el método de potencia...")
    autovalor, autovector = obtener_principal_autovector(X)
    if autovalor is None or autovector is None:
        print("Error: No se pudo calcular el principal autovector y autovalor.")
        return
    print(f"Principal Autovalor: {autovalor}")
    print(f"Principal Autovector (primeros 10 elementos): {autovector[:10]}")
    
    # Reducción de dimensionalidad a k=1 (puedes ajustar 'k' según tus necesidades)
    k = 1
    print(f"\nReduciendo la dimensionalidad a k={k}...")
    
    # Seleccionar los primeros k elementos de V_k (autovector)
    V_k = autovector[:k]  # Lista de k elementos
    Sigma_k = autovalor
    
    # Calcular U_k = (X * V_k) / Sigma_k
    # Multiplicar X por V_k
    U_k = multiplicar_matriz_discreta_vector(X, V_k)  # Lista de m elementos
    # Normalizar U_k
    U_k = [x / Sigma_k for x in U_k]
    
    # Proyectar términos y documentos en el espacio reducido
    print("\nProyectando términos y documentos en el espacio reducido...")
    
    # Proyección de términos: U_k * Sigma_k
    terminos_proyectados = [u * Sigma_k for u in U_k]  # Lista de m elementos
    
    # Proyección de documentos: V_k * Sigma_k
    documentos_proyectados = [v * Sigma_k for v in V_k]  # Lista de k elementos
    
    # Mostrar resultados
    print(f"Documentos proyectados en {k} dimensión(es):")
    for idx, peso in enumerate(documentos_proyectados):
        print(f"Documento {idx+1}: {peso}")
    
    print(f"\nTérminos proyectados en {k} dimensión(es):")
    for term, peso in zip(sorted(vocabulario), terminos_proyectados):
        print(f"Término '{term}': {peso}")

if __name__ == "__main__":
    main()


Esta implementación utiliza el método de potencia para calcular el principal autovector y autovalor, lo cual es adecuado para conjuntos de datos pequeños. Sin embargo, no es eficiente para matrices grandes.
Para múltiples componentes ($k > 1$), se necesitaría una implementación más avanzada que calcule múltiples autovectores y autovalores, posiblemente utilizando técnicas de deflación.

La matriz TF-IDF se representa como una lista de diccionarios, donde cada diccionario corresponde a una fila (término) y mapea índices de documentos a pesos. Esto optimiza el almacenamiento y las operaciones al evitar almacenar ceros.
Las funciones de multiplicación de matrices y vectores están adaptadas para manejar esta representación dispersa.

Puedes ajustar el valor de `k` para reducir la dimensionalidad a más dimensiones si implementas la capacidad de manejar múltiples componentes.
Para mejorar la precisión y eficiencia, considera implementar técnicas adicionales como la normalización de la matriz TF-IDF o el uso de otros métodos de descomposición.



### Word2vec

Código completo de Word2vec implementado en Python utilizando únicamente las librerías estándar y numpy.

In [ ]:
import re
import string
from collections import defaultdict
import numpy as np
import math
import random

# 1. Definición de Stopwords en español
STOPWORDS = {
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por',
    'un', 'para', 'con', 'no', 'una', 'su', 'al', 'es', 'lo', 'como', 'más',
    'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre',
    'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde',
    'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni',
    'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes',
    'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa',
    'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella',
    'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te',
    'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os',
    'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo',
    'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras',
    'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy',
    'estás', 'está', 'estamos', 'estáis', 'están', 'esté', 'estés', 'estemos',
    'estéis', 'estén', 'estaré', 'estarás', 'estará', 'estaremos', 'estaréis',
    'estarán', 'estaría', 'estarías', 'estaríamos', 'estaríais', 'estarían',
    'estaba', 'estabas', 'estábamos', 'estabais', 'estaban', 'estuve',
    'estuviste', 'estuvo', 'estuvimos', 'estuvisteis', 'estuvieron', 'estuviera',
    'estuvieras', 'estuviéramos', 'estuvierais', 'estuvieran', 'estuviese',
    'estuvieses', 'estuviésemos', 'estuvieseis', 'estuviesen', 'estando',
    'estado', 'estada', 'estados', 'estadas', 'estad'
}

# 2. Funciones de preprocesamiento

def tokenize(text):
    """
    Tokeniza el texto dividiéndolo en palabras, manejando puntuación y mayúsculas.
    """
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres de puntuación por espacios
    text = re.sub(f'[{re.escape(string.punctuation)}]', ' ', text)
    # Reemplazar dígitos por espacios
    text = re.sub(r'\d+', ' ', text)
    # Dividir en palabras
    tokens = text.split()
    return tokens

def simple_stem(word):
    """
    Realiza un stemming básico eliminando sufijos comunes en español.
    Esta implementación es muy simplificada y puede no ser precisa.
    """
    suffixes = ['ando', 'iendo', 'ar', 'er', 'ir', 'ado', 'ido', 'ción', 'sión', 'mente']
    for suffix in suffixes:
        if word.endswith(suffix) and len(word) > len(suffix) + 2:
            return word[:-len(suffix)]
    return word

def preprocess_corpus(file_path, min_freq=1):
    """
    Preprocesa el corpus:
    - Tokenización
    - Stemming
    - Remoción de stopwords
    - Filtrado de palabras raras
    """
    word_freq = defaultdict(int)
    processed_tokens = []

    print("Iniciando el preprocesamiento del corpus...")

    # Primera pasada: contar frecuencias
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            tokens = tokenize(line)
            tokens = [simple_stem(token) for token in tokens]
            tokens = [token for token in tokens if token not in STOPWORDS]
            for token in tokens:
                word_freq[token] += 1

    print("Frecuencias de palabras calculadas.")

    # Filtrar palabras raras
    vocab = {word for word, freq in word_freq.items() if freq >= min_freq}
    print(f"Vocabulario filtrado a {len(vocab)} palabras.")

    # Segunda pasada: recolectar tokens válidos
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            tokens = tokenize(line)
            tokens = [simple_stem(token) for token in tokens]
            tokens = [token for token in tokens if token not in STOPWORDS and token in vocab]
            processed_tokens.extend(tokens)

    print("Preprocesamiento completado.")
    return processed_tokens, vocab

# 3. Construcción del vocabulario y generación de datos de entrenamiento

def build_vocab(tokens):
    """
    Construye un mapeo de palabras a índices y viceversa.
    """
    word_to_ix = {word: idx for idx, word in enumerate(set(tokens))}
    ix_to_word = {idx: word for word, idx in word_to_ix.items()}
    return word_to_ix, ix_to_word

def generate_training_data_cbow(tokens, word_to_ix, window_size=2):
    """
    Genera pares de (contexto, palabra objetivo) para CBOW.
    """
    training_data = []
    for i in range(len(tokens)):
        target = word_to_ix[tokens[i]]
        context = []
        for j in range(-window_size, window_size + 1):
            if j != 0 and 0 <= i + j < len(tokens):
                context.append(word_to_ix[tokens[i + j]])
        training_data.append((context, target))
    return training_data

def generate_training_data_skipgram(tokens, word_to_ix, window_size=2):
    """
    Genera pares de (palabra objetivo, contexto) para Skip-Gram.
    """
    training_data = []
    for i in range(len(tokens)):
        target = word_to_ix[tokens[i]]
        context = []
        for j in range(-window_size, window_size + 1):
            if j != 0 and 0 <= i + j < len(tokens):
                context.append(word_to_ix[tokens[i + j]])
        training_data.append((target, context))
    return training_data

# 4. Implementación de los modelos Word2vec

class CBOW:
    def __init__(self, vocab_size, embedding_dim):
        """
        Inicializa los vectores de embeddings.
        """
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        # Vectores de entrada (W)
        self.W = np.random.uniform(-0.5 / embedding_dim, 0.5 / embedding_dim, (vocab_size, embedding_dim))
        # Vectores de salida (W')
        self.W_prime = np.random.uniform(-0.5 / embedding_dim, 0.5 / embedding_dim, (vocab_size, embedding_dim))
    
    def forward(self, context_indices):
        """
        Calcula la representación del contexto.
        """
        # Promedio de los vectores de contexto
        v_context = np.mean(self.W[context_indices], axis=0)
        return v_context
    
    def predict(self, v_context):
        """
        Calcula las puntuaciones para todas las palabras en el vocabulario.
        """
        scores = np.dot(self.W_prime, v_context)
        return scores
    
    def softmax(self, scores):
        """
        Calcula la distribución de probabilidad usando softmax.
        """
        exp_scores = np.exp(scores - np.max(scores))  # Estabilidad numérica
        return exp_scores / np.sum(exp_scores)
    
    def train(self, training_data, epochs=5, learning_rate=0.05):
        """
        Entrena el modelo CBOW usando descenso de gradiente.
        """
        for epoch in range(epochs):
            loss = 0
            for context, target in training_data:
                # Forward pass
                v_context = self.forward(context)
                scores = self.predict(v_context)
                probs = self.softmax(scores)
                
                # Calculando la pérdida (-log probabilidad del objetivo)
                loss += -math.log(probs[target] + 1e-7)
                
                # Gradiente de la pérdida respecto a scores
                d_scores = probs.copy()
                d_scores[target] -= 1  # y_hat - y
                
                # Gradiente respecto a W'
                dW_prime = np.outer(d_scores, v_context)
                
                # Gradiente respecto a v_context
                dv_context = np.dot(self.W_prime.T, d_scores)
                dv_context /= len(context)  # Promedio
                
                # Actualización de W' y W
                self.W_prime -= learning_rate * dW_prime
                for idx in context:
                    self.W[idx] -= learning_rate * dv_context
            
            print(f"Epoch {epoch + 1}/{epochs}, Pérdida: {loss:.4f}")

class SkipGram:
    def __init__(self, vocab_size, embedding_dim, K=5):
        """
        Inicializa los vectores de embeddings.
        """
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.K = K  # Número de muestras negativas
        # Vectores de entrada (W)
        self.W = np.random.uniform(-0.5 / embedding_dim, 0.5 / embedding_dim, (vocab_size, embedding_dim))
        # Vectores de salida (W')
        self.W_prime = np.random.uniform(-0.5 / embedding_dim, 0.5 / embedding_dim, (vocab_size, embedding_dim))
        # Pre-computar la distribución para Negative Sampling
        self.word_freq = defaultdict(int)
        self.neg_sampling_prob = None
    
    def build_negative_sampling_distribution(self, tokens, word_to_ix):
        """
        Construye una distribución para muestreo negativo basada en la frecuencia de palabras.
        Utiliza la fórmula de unigram raised to the 3/4 power.
        """
        for token in tokens:
            self.word_freq[word_to_ix[token]] += 1
        # Calcular la frecuencia normalizada
        self.neg_sampling_prob = np.array([self.word_freq[i]**0.75 for i in range(self.vocab_size)])
        self.neg_sampling_prob /= np.sum(self.neg_sampling_prob)
    
    def sample_negative(self, exclude, num_samples):
        """
        Muestra palabras negativas, excluyendo ciertas palabras.
        """
        negatives = []
        while len(negatives) < num_samples:
            sampled = np.random.choice(self.vocab_size, p=self.neg_sampling_prob)
            if sampled not in exclude:
                negatives.append(sampled)
        return negatives
    
    def sigmoid(self, x):
        """
        Calcula la función sigmoide.
        """
        return 1 / (1 + np.exp(-x))
    
    def train(self, training_data, epochs=5, learning_rate=0.05):
        """
        Entrena el modelo Skip-Gram con Negative Sampling.
        """
        for epoch in range(epochs):
            loss = 0
            for target, context in training_data:
                # Para cada palabra de contexto
                for context_word in context:
                    # Positivo
                    z = np.dot(self.W_prime[context_word], self.W[target])
                    sigma = self.sigmoid(z)
                    loss += -math.log(sigma + 1e-7)
                    grad_z = sigma - 1
                    # Gradientes
                    grad_W_prime = grad_z * self.W[target]
                    grad_W = grad_z * self.W_prime[context_word]
                    # Actualizar W' y W
                    self.W_prime[context_word] -= learning_rate * grad_W_prime
                    self.W[target] -= learning_rate * grad_W
                    
                    # Negative Sampling
                    negatives = self.sample_negative(exclude={context_word}, num_samples=self.K)
                    for negative in negatives:
                        z_neg = np.dot(self.W_prime[negative], self.W[target])
                        sigma_neg = self.sigmoid(-z_neg)
                        loss += -math.log(sigma_neg + 1e-7)
                        grad_z_neg = sigma_neg  # derivada de -log(sigmoid(-z_neg)) respecto a z_neg
                        # Gradientes
                        grad_W_prime_neg = grad_z_neg * self.W[target]
                        grad_W_neg = grad_z_neg * self.W_prime[negative]
                        # Actualizar W' y W
                        self.W_prime[negative] -= learning_rate * grad_W_prime_neg
                        self.W[target] -= learning_rate * grad_W_neg
            print(f"Epoch {epoch + 1}/{epochs}, Pérdida: {loss:.4f}")

# 5. Función principal para ejecutar el modelo

def main():
    # 1. Ruta al archivo de corpus
    corpus_path = 'corpus_es.txt'  # Asegúrate de tener este archivo en el mismo directorio
    
    # 2. Preprocesamiento
    tokens, vocabulario = preprocess_corpus(corpus_path, min_freq=1)  # Cambiado a min_freq=1 para el ejemplo
    print(f"Total de tokens después del preprocesamiento: {len(tokens)}")
    print(f"Tamaño del vocabulario: {len(vocabulario)}")
    
    # 3. Construir mapeos
    word_to_ix, ix_to_word = build_vocab(tokens)
    
    # 4. Generar datos de entrenamiento para CBOW
    training_data_cbow = generate_training_data_cbow(tokens, word_to_ix, window_size=2)
    
    # 5. Entrenar CBOW
    print("\nEntrenando el modelo CBOW...")
    cbow_model = CBOW(vocab_size=len(word_to_ix), embedding_dim=100)
    cbow_model.train(training_data_cbow, epochs=5, learning_rate=0.05)
    
    # 6. Generar datos de entrenamiento para Skip-Gram
    training_data_skipgram = generate_training_data_skipgram(tokens, word_to_ix, window_size=2)
    
    # 7. Entrenar Skip-Gram con Negative Sampling
    print("\nEntrenando el modelo Skip-Gram con Negative Sampling...")
    skipgram_model = SkipGram(vocab_size=len(word_to_ix), embedding_dim=100, K=5)
    skipgram_model.build_negative_sampling_distribution(tokens, word_to_ix)
    skipgram_model.train(training_data_skipgram, epochs=5, learning_rate=0.05)
    
    # 8. Guardar los embeddings
    np.save('cbow_embeddings_W.npy', cbow_model.W)
    np.save('cbow_embeddings_W_prime.npy', cbow_model.W_prime)
    np.save('skipgram_embeddings_W.npy', skipgram_model.W)
    np.save('skipgram_embeddings_W_prime.npy', skipgram_model.W_prime)
    print("\nEmbeddings guardados exitosamente.")
    
    # 9. Guardar el mapeo de índices a palabras
    with open('ix_to_word.txt', 'w', encoding='utf-8') as f:
        for idx in sorted(ix_to_word.keys()):
            f.write(f"{idx}\t{ix_to_word[idx]}\n")
    print("Mapa de índices a palabras guardado exitosamente.")

if __name__ == "__main__":
    main()


El código presenta una versión donde el `min_freq` se establece en 1. Esto garantiza que incluso con el corpus de ejemplo pequeño, obtendrás un vocabulario no vacío. 

**Observación:**

Un problema a afrontar con un corpus de ejemplo pequeño y un umbral de frecuencia alto (`min_freq=5`), es que ninguna palabra cumplió con el criterio para ser incluida en el vocabulario. Al reducir `min_freq` a 1 o ampliar el corpus para que las palabras clave se repitan al menos 5 veces, se resuelve el problema y se permite que el modelo entrenara correctamente.

In [ ]:
import re
import math
import random
from collections import defaultdict, Counter

# 1. Definición de stopwords

STOPWORDS_ES = {
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se',
    'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al',
    'es', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o',
    'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy',
    'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde',
    'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno',
    'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos',
    'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo',
    'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho',
    'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar',
    'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú',
    'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros',
    'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo',
    'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas',
    'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro',
    'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy',
    'estás', 'está', 'estamos', 'estáis', 'están', 'esté',
    'estés', 'estemos', 'estéis', 'estén', 'estaré', 'estarás',
    'estará', 'estaremos', 'estaréis', 'estarán', 'estaría',
    'estarías', 'estaríamos', 'estaríais', 'estarían', 'estaba',
    'estabas', 'estábamos', 'estabais', 'estaban', 'estuve',
    'estuviste', 'estuvo', 'estuvimos', 'estuvisteis', 'estuvieron',
    'estuviera', 'estuvieras', 'estuviéramos', 'estuvierais',
    'estuvieran', 'estuviese', 'estuvieses', 'estuviésemos',
    'estuvieseis', 'estuviesen', 'estando', 'estado', 'estada',
    'estados', 'estadas', 'estad'
}


# 2. Funciones de preprocesamiento


def tokenize(text):
    """
    Tokeniza el texto: convierte a minúsculas, elimina puntuación y divide en palabras.
    """
    text = text.lower()
    # Reemplazar caracteres no alfabéticos por espacios
    text = re.sub(r'[^a-záéíóúüñ\s]', ' ', text)
    # Dividir por espacios
    tokens = text.split()
    return tokens

def simple_stem(word):
    """
    Realiza un stemming muy básico eliminando sufijos comunes en español.
    """
    suffixes = ['ando', 'iendo', 'ar', 'er', 'ir', 'ado', 'ido', 'es', 'as', 'os', 'a', 'e', 'o']
    for suffix in suffixes:
        if word.endswith(suffix) and len(word) > len(suffix) + 2:
            return word[:-len(suffix)]
    return word

def preprocess_corpus(file_path, stopwords, min_freq=5):
    """
    Preprocesa el corpus: tokenización, stemming, eliminación de stopwords y filtrado de palabras raras.
    
    Args:
        file_path (str): Ruta al archivo de texto del corpus.
        stopwords (set): Conjunto de palabras vacías en español.
        min_freq (int): Umbral de frecuencia mínima para incluir una palabra en el vocabulario.
    
    Returns:
        list: Lista de palabras preprocesadas.
        dict: Mapeo de palabras a índices.
    """
    word_counts = Counter()
    corpus = []
    
    print("Iniciando preprocesamiento del corpus...")
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            tokens = tokenize(line)
            tokens = [simple_stem(token) for token in tokens]
            tokens = [token for token in tokens if token not in stopwords]
            corpus.extend(tokens)
            word_counts.update(tokens)
            
            if line_num % 10000 == 0:
                print(f"Procesadas {line_num} líneas...")
    
    print("Filtrando palabras raras...")
    # Filtrar palabras raras
    corpus = [word for word in corpus if word_counts[word] >= min_freq]
    
    # Crear vocabulario
    vocab = {word: idx for idx, (word, count) in enumerate(word_counts.items()) if count >= min_freq}
    
    print(f'Tamaño del vocabulario después del filtrado: {len(vocab)}')
    return corpus, vocab


# 3. Construcción de la matriz de co-ocurrencia


def build_cooccurrence_matrix(corpus, vocab, window_size=5):
    """
    Construye la matriz de co-ocurrencia.
    
    Args:
        corpus (list): Lista de palabras preprocesadas.
        vocab (dict): Mapeo de palabras a índices.
        window_size (int): Tamaño de la ventana de contexto.
    
    Returns:
        dict: Matriz de co-ocurrencia representada como diccionario de diccionarios.
    """
    cooc_matrix = defaultdict(lambda: defaultdict(int))
    corpus_length = len(corpus)
    
    print("Construyendo la matriz de co-ocurrencia...")
    for idx, word in enumerate(corpus):
        word_id = vocab[word]
        start = max(idx - window_size, 0)
        end = min(idx + window_size + 1, corpus_length)
        for i in range(start, end):
            if i != idx:
                context_word = corpus[i]
                context_id = vocab[context_word]
                cooc_matrix[word_id][context_id] += 1
                
        if (idx + 1) % 100000 == 0:
            print(f"Procesadas {idx + 1} palabras del corpus...")
    
    print("Matriz de co-ocurrencia construida.")
    return cooc_matrix


# 4. Implementación del modelo GloVe


def initialize_parameters(vocab_size, embedding_dim):
    """
    Inicializa los vectores de palabras, vectores de contexto y sesgos.
    
    Args:
        vocab_size (int): Número de palabras en el vocabulario.
        embedding_dim (int): Dimensionalidad de los embeddings.
    
    Returns:
        dict, dict, list, list: Vectores de palabras, vectores de contexto, sesgos de palabras, sesgos de contexto.
    """
    W = {i: [random.uniform(-0.5, 0.5) for _ in range(embedding_dim)] for i in range(vocab_size)}
    W_tilde = {i: [random.uniform(-0.5, 0.5) for _ in range(embedding_dim)] for i in range(vocab_size)}
    b = [0.0 for _ in range(vocab_size)]
    b_tilde = [0.0 for _ in range(vocab_size)]
    return W, W_tilde, b, b_tilde

def weighting_function(x, x_max=100, alpha=0.75):
    if x < x_max:
        return (x / x_max) ** alpha
    else:
        return 1.0

def train_glove(cooc_matrix, vocab_size, embedding_dim=50, x_max=100, alpha=0.75, learning_rate=0.05, epochs=25):
    """
    Entrena el modelo GloVe.
    
    Args:
        cooc_matrix (dict): Matriz de co-ocurrencia.
        vocab_size (int): Tamaño del vocabulario.
        embedding_dim (int): Dimensionalidad de los embeddings.
        x_max (int): Parámetro para la función de ponderación.
        alpha (float): Parámetro para la función de ponderación.
        learning_rate (float): Tasa de aprendizaje.
        epochs (int): Número de épocas de entrenamiento.
    
    Returns:
        dict: Vectores de palabras entrenados.
    """
    W, W_tilde, b, b_tilde = initialize_parameters(vocab_size, embedding_dim)
    
    print("Iniciando entrenamiento del modelo GloVe...")
    for epoch in range(1, epochs + 1):
        total_cost = 0.0
        count = 0
        for i, context_dict in cooc_matrix.items():
            for j, X_ij in context_dict.items():
                f_X_ij = weighting_function(X_ij, x_max, alpha)
                log_X_ij = math.log(X_ij)
                
                # Predicción
                dot = sum([W[i][k] * W_tilde[j][k] for k in range(embedding_dim)])
                prediction = dot + b[i] + b_tilde[j]
                
                # Error
                error = prediction - log_X_ij
                cost = f_X_ij * (error ** 2)
                total_cost += cost
                count +=1
                
                # Gradientes
                grad_common = 2 * f_X_ij * error
                for k in range(embedding_dim):
                    W[i][k] -= learning_rate * (grad_common * W_tilde[j][k])
                    W_tilde[j][k] -= learning_rate * (grad_common * W[i][k])
                b[i] -= learning_rate * grad_common
                b_tilde[j] -= learning_rate * grad_common
                
            if i % 10000 == 0:
                print(f"Procesadas {i} palabras en la época {epoch}...")
        
        avg_cost = total_cost / count if count != 0 else 0
        print(f'Época {epoch}/{epochs}, Costo Promedio: {avg_cost}')
    
    # Combinar W y W_tilde para obtener los embeddings finales
    embeddings = {}
    for i in range(vocab_size):
        embeddings[i] = [W[i][k] + W_tilde[i][k] for k in range(embedding_dim)]
    
    print("Entrenamiento completado.")
    return embeddings

# 5. Evaluación de los embeddings


def find_analogy(word_a, word_b, word_c, embeddings, vocab, index_to_word, top_n=1):
    """
    Resuelve analogías del tipo: word_a es a word_b como word_c es a ?
    
    Args:
        word_a (str): Primera palabra de la analogía.
        word_b (str): Segunda palabra de la analogía.
        word_c (str): Tercera palabra de la analogía.
        embeddings (dict): Vectores de palabras.
        vocab (dict): Mapeo de palabras a índices.
        index_to_word (dict): Mapeo de índices a palabras.
        top_n (int): Número de resultados a retornar.
    
    Returns:
        list: Lista de palabras que completan la analogía.
    """
    if word_a not in vocab or word_b not in vocab or word_c not in vocab:
        print('Una de las palabras no está en el vocabulario.')
        return []
    
    vec_a = embeddings[vocab[word_a]]
    vec_b = embeddings[vocab[word_b]]
    vec_c = embeddings[vocab[word_c]]
    
    # Calcular el vector objetivo
    target_vec = [vec_b[i] - vec_a[i] + vec_c[i] for i in range(len(vec_a))]
    
    # Calcular similitud coseno
    similarities = {}
    norm_target = math.sqrt(sum([x**2 for x in target_vec]))
    for idx, vec in embeddings.items():
        numerator = sum([target_vec[i] * vec[i] for i in range(len(vec))])
        norm_vec = math.sqrt(sum([x**2 for x in vec]))
        if norm_vec == 0:
            similarity = 0
        else:
            similarity = numerator / (norm_target * norm_vec)
        similarities[idx] = similarity
    
    # Ordenar y retornar las mejores coincidencias
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    
    # Excluir las palabras de entrada
    results = []
    for idx, sim in sorted_similarities:
        if idx not in [vocab[word_a], vocab[word_b], vocab[word_c]]:
            results.append(index_to_word[idx])
            if len(results) == top_n:
                break
    return results

# 6. Función principal


def main():
    corpus_file = 'corpus1.txt'  # Ruta al archivo del corpus
    min_frequency = 1          # Umbral de frecuencia mínima
    window_size = 5            # Tamaño de la ventana de contexto
    embedding_dim = 50         # Dimensionalidad de los embeddings
    epochs = 25                # Número de épocas de entrenamiento
    
    # Preprocesamiento del corpus
    corpus, vocab = preprocess_corpus(corpus_file, STOPWORDS_ES, min_freq=min_frequency)
    
    # Construcción de la matriz de co-ocurrencia
    cooc_matrix = build_cooccurrence_matrix(corpus, vocab, window_size=window_size)
    
    # Entrenamiento del modelo GloVe
    embeddings = train_glove(cooc_matrix, len(vocab), embedding_dim=embedding_dim, epochs=epochs)
    
    # Crear mapeo de índices a palabras
    index_to_word = {idx: word for word, idx in vocab.items()}
    
    # Ejemplo de analogía: 'rey' es a 'reina' como 'hombre' es a 'mujer'
    # Nota: Asegúrate de que estas palabras existan en tu vocabulario después del preprocesamiento
    analogy_words = ('re', 'reina', 'hombre')  # 'rey' se puede haber convertido a 're' por stemming
    word_a, word_b, word_c = analogy_words
    analogy_result = find_analogy(word_a, word_b, word_c, embeddings, vocab, index_to_word)
    if analogy_result:
        print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a '{analogy_result[0]}'")
    else:
        print("No se pudo encontrar una palabra para completar la analogía.")

if __name__ == "__main__":
    main()


Este código puede no ser eficiente para corpus muy grandes como el de Wikipedia. Para manejar grandes volúmenes de datos, se recomienda optimizar el almacenamiento de la matriz de co-ocurrencia, posiblemente utilizando estructuras de datos más eficientes o almacenándola en disco.
El entrenamiento del modelo GloVe con descenso de gradiente básico puede ser muy lento para vocabularios grandes. Implementaciones más eficientes podrían utilizar métodos de optimización avanzados como AdaGrad.